In [1575]:
import math

import plotly.graph_objects as go
import plotly.offline as offline
import os
from datetime import datetime
import yaml

def hsv_to_rgb(h, s, v):
    h_i = int(h * 6)
    f = h * 6 - h_i
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    if h_i == 0:
        return v, t, p
    elif h_i == 1:
        return q, v, p
    elif h_i == 2:
        return p, v, t
    elif h_i == 3:
        return p, q, v
    elif h_i == 4:
        return t, p, v
    else:
        return v, p, q


def generate_colors(num_colors):
    colors = []
    for i in range(num_colors):
        hue = i * (360.0 / num_colors)
        rgb = hsv_to_rgb(hue / 360.0, 1.0, 1.0)
        if i == 0:
            colors.append('rgb({},{},{})'.format(int(211), int(211), int(211)))
        elif i == 1:
            colors.append('rgb({},{},{})'.format(int(128), int(0), int(128)))
        elif i == 2:
            colors.append('rgb({},{},{})'.format(int(0), int(255), int(0)))
        elif i == 3:
            colors.append('rgb({},{},{})'.format(int(255), int(0), int(0)))
        elif i == 4:
            colors.append('rgb({},{},{})'.format(int(0), int(0), int(255)))
        else:
            colors.append('rgb({},{},{})'.format(int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)))
    return colors


def plot_cube_plotly(cube_definition, container_def=None):
    fig = go.Figure()

    if container_def is not None:
        x, y, z, l, w, h = 0, 0, 0, container_def[0], container_def[1], container_def[2]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color="gray", opacity=0.2, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))

    color_list = generate_colors(len(cube_definition))
    annotations_list = []
    for i, cube in enumerate(cube_definition):
        l, w, h, x, y, z = cube[:6]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        if len(cube)>6:
            annotations_list.append([text_x, text_y, text_z, str(cube[6])])
        else:
            annotations_list.append([text_x, text_y, text_z, str(i + 1)])
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color=color_list[i], opacity=0.8, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))

    fig.update_layout(
        scene=dict(
            aspectmode="data",
            annotations=[
                dict(
                    showarrow=False,
                    x=it[0],
                    y=it[1],
                    z=it[2],
                    text=it[3],
                    font=dict(
                        color="black",
                        size=14
                    ),
                    opacity=0.7) for it in annotations_list

            ]
        )
    )

    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%Y_%m_%d_%H_%M_%S')
    folder_path = f'./container_html/{date_str}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_name = f'container_{time_str}.html'
    file_path = os.path.join(folder_path, file_name)
    offline.plot(fig, filename=file_path, auto_open=True)
    # return HTML(fig.to_html())

def plot_cube_plotly(cube_definition, container_def=None, suctioncup=None):
    fig = go.Figure()

    if container_def is not None:
        x, y, z, l, w, h = 0, 0, 0, container_def[0], container_def[1], container_def[2]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        # 定义要显示的四个面（底面、前面、后面、左面）
        visible_faces = [
            [0, 1, 2, 3],  # 底面 (z=0)
            [0, 1, 5, 4],  # 前面 (y=0)
            [2, 3, 7, 6],  # 后面 (y=w)
            [0, 3, 7, 4]   # 左面 (x=0)
        ]
        
        # 添加四个可见面（灰色半透明）
        for face in visible_faces:
            vertices = [cube_definition_vertices[idx] for idx in face]
            x_vals = [v[0] for v in vertices]
            y_vals = [v[1] for v in vertices]
            z_vals = [v[2] for v in vertices]
            
            fig.add_trace(go.Mesh3d(
                x=x_vals,
                y=y_vals,
                z=z_vals,
                i=[0, 0, 0],  # 第一个三角形
                j=[1, 2, 3],  # 顶点索引
                k=[2, 3, 1],  # 顶点索引
                color='gray',
                opacity=0.5,
                showlegend=False
            ))
            
        # 定义12根边线（每根边由两个顶点组成）
        edges = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # 底面4根边
            [4, 5], [6, 7], [7, 4],  # 顶面4根边
            [0, 4], [1, 5], [2, 6], [3, 7]   # 4根垂直边
        ]
        
        # 添加所有边线（黑色实线）
        for edge in edges:
            start, end = edge
            x_line = [cube_definition_vertices[start][0], cube_definition_vertices[end][0]]
            y_line = [cube_definition_vertices[start][1], cube_definition_vertices[end][1]]
            z_line = [cube_definition_vertices[start][2], cube_definition_vertices[end][2]]
            
            fig.add_trace(go.Scatter3d(
                x=x_line,
                y=y_line,
                z=z_line,
                mode='lines',
                line=dict(color='black', width=3),
                showlegend=False
            ))

    color_list = generate_colors(len(cube_definition))
    annotations_list = []
    for i, cube in enumerate(cube_definition):
        l, w, h, x, y, z = cube[:6]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        if len(cube)>6:
            annotations_list.append([text_x, text_y, text_z, str(cube[6])])
        else:
            annotations_list.append([text_x, text_y, text_z, str(i + 1)])
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color=color_list[i], opacity=0.8, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))

    if suctioncup is not None:
        x, y, z, l, w, h = suctioncup[0], suctioncup[1], suctioncup[2], suctioncup[3]-suctioncup[0], suctioncup[4]-suctioncup[1], suctioncup[5]-suctioncup[2]
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        annotations_list.append([text_x, text_y, text_z, '端拾器'])
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color="black", opacity=0.2, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))
        
    
    fig.update_layout(
        scene=dict(
            aspectmode="data",
            annotations=[
                dict(
                    showarrow=False,
                    x=it[0],
                    y=it[1],
                    z=it[2],
                    text=it[3],
                    font=dict(
                        color="black",
                        size=14
                    ),
                    opacity=0.7) for it in annotations_list

            ]
        )
    )

    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%Y_%m_%d_%H_%M_%S')
    folder_path = f'./container_html/{date_str}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_name = f'container_{time_str}.html'
    file_path = os.path.join(folder_path, file_name)
    offline.plot(fig, filename=file_path, auto_open=True)

if __name__ == '__main__':
    # container_package_list =  [{'x': 10.152532196044922, 'y': 8.963372802734375, 'z': 4.825049209594726, 'length': 16.054150390625, 'width': 18.79766387939453, 'height': 10.340033721923827, 'dw': 0.9998429417610168, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0183824480498-1-1-'}, {'x': 7.0130760192871096, 'y': 34.996466064453124, 'z': 4.159442138671875, 'length': 14.398129272460938, 'width': 13.989132690429688, 'height': 9.044147491455078, 'dw': 0.9997024536132812, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0183830270532-2-2-'}, {'x': 7.884239196777344, 'y': 33.39789733886719, 'z': 12.158470916748048, 'length': 16.95948181152344, 'width': 21.328265380859374, 'height': 8.947252655029297, 'dw': 0.9997333884239197, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0183827673390-1-2-'}, {'x': 6.816968536376953, 'y': 30.198410034179688, 'z': 21.055657958984376, 'length': 14.721957397460937, 'width': 16.98345489501953, 'height': 10.326156616210938, 'dw': -0.9990259408950806, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19104725217-1-1-'}, {'x': 9.138644409179687, 'y': 28.85126647949219, 'z': 30.8895263671875, 'length': 14.404020690917969, 'width': 16.558367919921874, 'height': 11.084098815917969, 'dw': 0.9998131394386292, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038192553302-1-1-'}, {'x': 6.057394790649414, 'y': 8.016008758544922, 'z': 14.1197509765625, 'length': 12.729918670654296, 'width': 16.917276000976564, 'height': 9.079509735107422, 'dw': -0.9983617067337036, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038188839189-1-1-'}]

    # {'package': [{'id': 'JD0207789560537-1-1-', 'length': 16.976580810546874, 'width': 9.9, 'height': 10.7, 'weight': 0.98, 'end_machine_code': 'BJHCFJ-CZ-ZDFJJ-SZJL-LH', 'end_chute_code': '33', 'end_site_code': '866468', 'end_site_name': '北京四季怡园小站', 'predicted_alloc_time': '2025-09-22 12:32:59.936', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': True, 'pallet_id': [1], 'grasp_valid': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'grasp_points_delta': [{'delta_x': 86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 86.45402526855469, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -9.606002807617188, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -67.24201965332031, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 67.24201965332031, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 28.81800079345703, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -28.818012237548828, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 1.1371326446533203, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 3.4113974571228027, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -3.4113969802856445, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -5.685662269592285, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -7.959926605224609, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 7.959927082061768, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': -48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': 10.23419189453125, 'delta_z': 0.0}, {'delta_x': 48.03001403808594, 'delta_y': -10.23419189453125, 'delta_z': 0.0}]}, {'id': 'JDV023062968590-1-1-', 'length': 13.45, 'width': 12.45, 'height': 6.8, 'weight': 3.86, 'end_machine_code': 'BJHCFJ-CZ-ZDFJJ-SZJL-LH', 'end_chute_code': '31', 'end_site_code': '288696', 'end_site_name': '北京曦景长安站', 'predicted_alloc_time': '2025-09-22 12:37:58.688', 'al
    container_package_list = [{'x': 21.86756286621094, 'y': 19.944813537597657, 'z': 2.1876195907592773, 'length': 47.72745056152344, 'width': 44.30244140625, 'height': 5.106596755981445, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '437.351257'}]
    
    log_packages = []
    log_packages = [
        [p['length'], p['width'], p['height'], p['x']-p['length']/2, p['y']-p['width']/2, p['z']-p['height']/2] for p in container_package_list
    ]

    plot_cube_plotly(log_packages,[47,30,40])#[47,40,80]
    print('finished! ')


finished! 


In [13]:
# ============================================================
# MCTS-PCT 码垛结果可视化 (完整版)
# ============================================================
import json
import os
from datetime import datetime
import plotly.graph_objects as go
import plotly.offline as offline

# ============================================================
# 可视化函数定义
# ============================================================
def hsv_to_rgb(h, s, v):
    h_i = int(h * 6)
    f = h * 6 - h_i
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    if h_i == 0: return v, t, p
    elif h_i == 1: return q, v, p
    elif h_i == 2: return p, v, t
    elif h_i == 3: return p, q, v
    elif h_i == 4: return t, p, v
    else: return v, p, q

def generate_colors(num_colors):
    colors = []
    for i in range(num_colors):
        hue = i * (360.0 / max(num_colors, 1))
        rgb = hsv_to_rgb(hue / 360.0, 0.7, 0.9)
        colors.append('rgb({},{},{})'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255)))
    return colors

def plot_cube_plotly(cube_definition, container_def=None, save_html=True):
    fig = go.Figure()

    # 绘制容器
    if container_def is not None:
        x, y, z, l, w, h = 0, 0, 0, container_def[0], container_def[1], container_def[2]
        vertices = [[x,y,z], [x+l,y,z], [x+l,y+w,z], [x,y+w,z],
                    [x,y,z+h], [x+l,y,z+h], [x+l,y+w,z+h], [x,y+w,z+h]]
        fig.add_trace(go.Mesh3d(x=[v[0] for v in vertices], y=[v[1] for v in vertices], z=[v[2] for v in vertices],
            i=[0,0,4,4,1,5,0,4,1,1,2,6], j=[1,2,5,6,2,2,3,3,5,4,3,3], k=[2,3,6,7,5,6,4,7,4,0,6,7],
            color="gray", opacity=0.2, showlegend=False))
        edges = [[vertices[i] for i in [0,1,2,3,0]], [vertices[i] for i in [4,5,6,7,4]],
                 [vertices[0],vertices[4]], [vertices[1],vertices[5]], [vertices[2],vertices[6]], [vertices[3],vertices[7]]]
        for edge in edges:
            ex, ey, ez = zip(*edge)
            fig.add_trace(go.Scatter3d(x=ex, y=ey, z=ez, mode='lines', line=dict(color='black', width=2), showlegend=False))

    # 收集标号注释
    annotations_list = []
    
    # 绘制箱子
    color_list = generate_colors(len(cube_definition))
    for i, cube in enumerate(cube_definition):
        l, w, h, x, y, z = cube[:6]
        
        # 计算箱子中心位置（用于放置标号）
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        label = str(cube[6]) if len(cube) > 6 else str(i + 1)
        annotations_list.append([text_x, text_y, text_z, label])
        
        vertices = [[x,y,z], [x+l,y,z], [x+l,y+w,z], [x,y+w,z],
                    [x,y,z+h], [x+l,y,z+h], [x+l,y+w,z+h], [x,y+w,z+h]]
        fig.add_trace(go.Mesh3d(x=[v[0] for v in vertices], y=[v[1] for v in vertices], z=[v[2] for v in vertices],
            i=[0,0,4,4,1,5,0,4,1,1,2,6], j=[1,2,5,6,2,2,3,3,5,4,3,3], k=[2,3,6,7,5,6,4,7,4,0,6,7],
            color=color_list[i], opacity=0.8, showlegend=False))
        edges = [[vertices[i] for i in [0,1,2,3,0]], [vertices[i] for i in [4,5,6,7,4]],
                 [vertices[0],vertices[4]], [vertices[1],vertices[5]], [vertices[2],vertices[6]], [vertices[3],vertices[7]]]
        for edge in edges:
            ex, ey, ez = zip(*edge)
            fig.add_trace(go.Scatter3d(x=ex, y=ey, z=ez, mode='lines', line=dict(color='black', width=1), showlegend=False))

    # 添加标号到场景
    fig.update_layout(
        scene=dict(
            aspectmode="data",
            annotations=[
                dict(
                    showarrow=False,
                    x=it[0], y=it[1], z=it[2],
                    text=it[3],
                    font=dict(color="black", size=14),
                    opacity=0.9
                ) for it in annotations_list
            ]
        )
    )
    
    # 保存HTML文件
    if save_html:
        date_str = datetime.now().strftime('%Y-%m-%d')
        time_str = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        folder_path = f'./container_html/{date_str}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = os.path.join(folder_path, f'container_{time_str}.html')
        offline.plot(fig, filename=file_path, auto_open=True)
        print(f"HTML已保存到: {file_path}")
    else:
        fig.show()

# ============================================================
# 加载并可视化
# ============================================================
json_path = './mcts_results/mcts_results/2026-01-18/episode_2_00_55_45.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

cube_definition = data['cube_definition']
container_size = data['container_size']

stats = data['statistics']
print(f"Episode: {data['episode'] + 1}")
print(f"装入物品: {stats['packed_count']}/{stats['total_items']}")  
print(f"空间利用率: {stats['space_ratio']*100:.2f}%")
print(f"容器尺寸: {container_size}")

# 调用可视化（会自动保存HTML）
plot_cube_plotly(cube_definition, container_size, save_html=True)

Episode: 3
装入物品: 73/100
空间利用率: 50.93%
容器尺寸: [58, 56, 80]
HTML已保存到: ./container_html/2026-01-18\container_2026_01_18_22_03_08.html


In [10]:
def plot_cube_plotly(cube_definition, container_def=None, suctioncup=None):
    fig = go.Figure()

    if container_def is not None:
        x, y, z, l, w, h = 0, 0, 0, container_def[0], container_def[1], container_def[2]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        # 定义要显示的四个面（底面、前面、后面、左面）
        visible_faces = [
            [0, 1, 2, 3],  # 底面 (z=0)
            [0, 1, 5, 4],  # 前面 (y=0)
            [2, 3, 7, 6],  # 后面 (y=w)
            [0, 3, 7, 4]   # 左面 (x=0)
        ]
        
        # 添加四个可见面（灰色半透明）
        for face in visible_faces:
            vertices = [cube_definition_vertices[idx] for idx in face]
            x_vals = [v[0] for v in vertices]
            y_vals = [v[1] for v in vertices]
            z_vals = [v[2] for v in vertices]
            
            fig.add_trace(go.Mesh3d(
                x=x_vals,
                y=y_vals,
                z=z_vals,
                i=[0, 0, 0],  # 第一个三角形
                j=[1, 2, 3],  # 顶点索引
                k=[2, 3, 1],  # 顶点索引
                color='gray',
                opacity=0.5,
                showlegend=False
            ))
            
        # 定义12根边线（每根边由两个顶点组成）
        edges = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # 底面4根边
            [4, 5], [6, 7], [7, 4],  # 顶面4根边
            [0, 4], [1, 5], [2, 6], [3, 7]   # 4根垂直边
        ]
        
        # 添加所有边线（黑色实线）
        for edge in edges:
            start, end = edge
            x_line = [cube_definition_vertices[start][0], cube_definition_vertices[end][0]]
            y_line = [cube_definition_vertices[start][1], cube_definition_vertices[end][1]]
            z_line = [cube_definition_vertices[start][2], cube_definition_vertices[end][2]]
            
            fig.add_trace(go.Scatter3d(
                x=x_line,
                y=y_line,
                z=z_line,
                mode='lines',
                line=dict(color='black', width=3),
                showlegend=False
            ))

    color_list = generate_colors(len(cube_definition))
    annotations_list = []
    for i, cube in enumerate(cube_definition):
        l, w, h, x, y, z = cube[:6]
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        if len(cube)>6:
            annotations_list.append([text_x, text_y, text_z, str(cube[6])])
        else:
            annotations_list.append([text_x, text_y, text_z, str(i + 1)])
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color=color_list[i], opacity=0.8, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))

    if suctioncup is not None:
        x, y, z, l, w, h = suctioncup[0], suctioncup[1], suctioncup[2], suctioncup[3]-suctioncup[0], suctioncup[4]-suctioncup[1], suctioncup[5]-suctioncup[2]
        text_x = x + l / 2
        text_y = y + w / 2
        text_z = z + h / 2
        annotations_list.append([text_x, text_y, text_z, '端拾器'])
        cube_definition_vertices = [
            [x, y, z],
            [x + l, y, z],
            [x + l, y + w, z],
            [x, y + w, z],
            [x, y, z + h],
            [x + l, y, z + h],
            [x + l, y + w, z + h],
            [x, y + w, z + h]
        ]
        faces = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7]],
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[5],
             cube_definition_vertices[4]],
            [cube_definition_vertices[2], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[6]],
            [cube_definition_vertices[0], cube_definition_vertices[3], cube_definition_vertices[7],
             cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[2], cube_definition_vertices[6],
             cube_definition_vertices[5]]
        ]
        fig.add_trace(
            go.Mesh3d(
                x=[i[0] for i in cube_definition_vertices],
                y=[i[1] for i in cube_definition_vertices],
                z=[i[2] for i in cube_definition_vertices],
                i=[0, 0, 4, 4, 1, 5, 0, 4, 1, 1, 2, 6],
                j=[1, 2, 5, 6, 2, 2, 3, 3, 5, 4, 3, 3],
                k=[2, 3, 6, 7, 5, 6, 4, 7, 4, 0, 6, 7],
                color="black", opacity=0.2, showlegend=False
            )
        )
        edges = [
            [cube_definition_vertices[0], cube_definition_vertices[1], cube_definition_vertices[2],
             cube_definition_vertices[3], cube_definition_vertices[0]],
            [cube_definition_vertices[4], cube_definition_vertices[5], cube_definition_vertices[6],
             cube_definition_vertices[7], cube_definition_vertices[4]],
            [cube_definition_vertices[0], cube_definition_vertices[4]],
            [cube_definition_vertices[1], cube_definition_vertices[5]],
            [cube_definition_vertices[2], cube_definition_vertices[6]],
            [cube_definition_vertices[3], cube_definition_vertices[7]]
        ]
        for edge in edges:
            x, y, z = zip(*edge)
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='black'), showlegend=False))
        
    
    fig.update_layout(
        scene=dict(
            aspectmode="data",
            annotations=[
                dict(
                    showarrow=False,
                    x=it[0],
                    y=it[1],
                    z=it[2],
                    text=it[3],
                    font=dict(
                        color="black",
                        size=14
                    ),
                    opacity=0.7) for it in annotations_list

            ]
        )
    )

    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%Y_%m_%d_%H_%M_%S')
    folder_path = f'./container_html/{date_str}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_name = f'container_{time_str}.html'
    file_path = os.path.join(folder_path, file_name)
    offline.plot(fig, filename=file_path, auto_open=True)

In [ ]:
{'package': [{'id': 'JDVC29861661315-1-1-', 'length': 12.15, 'width': 12.05, 'height': 9.47669677734375, 'weight': 2.04, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '252942', 'end_site_name': '天津津门湖营业部', 'predicted_alloc_time': '2025-04-22 10:15:27.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': True, 'pallet_id': [2], 'grasp_valid': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'grasp_points_delta': [{'delta_x': -75.0, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': 75.0, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': -75.0, 'delta_y': 80.0, 'delta_z': 0.0}, {'delta_x': 75.0, 'delta_y': 80.0, 'delta_z': 0.0}, {'delta_x': -75.0, 'delta_y': -80.0, 'delta_z': 0.0}, {'delta_x': 75.0, 'delta_y': -80.0, 'delta_z': 0.0}, {'delta_x': -103.5, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': -103.5, 'delta_y': 80.0, 'delta_z': 0.0}, {'delta_x': -103.5, 'delta_y': -80.0, 'delta_z': 0.0}, {'delta_x': 103.5, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': 103.5, 'delta_y': 80.0, 'delta_z': 0.0}, {'delta_x': 103.5, 'delta_y': -80.0, 'delta_z': 0.0}, {'delta_x': -40.0, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': 40.0, 'delta_y': 0.0, 'delta_z': 0.0}, {'delta_x': -40.0, 'delta_y': 62.0, 'delta_z': 0.0}, {'delta_x': 40.0, 'delta_y': 62.0, 'delta_z': 0.0}, {'delta_x': -40.0, 'delta_y': -62.0, 'delta_z': 0.0}, {'delta_x': 40.0, 'delta_y': -62.0, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': -75.96612548828125, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': 75.96612548828125, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': 59.08476257324219, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': -59.08476257324219, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': -28.2025146484375, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': 28.2025146484375, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': -84.6075439453125, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': 84.6075439453125, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': -42.203399658203125, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': 42.20340347290039, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': 8.440681457519531, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': -8.440681457519531, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': -47.00419235229492, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': 47.00419616699219, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': -9.400840759277344, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': 9.400840759277344, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': -25.322044372558594, 'delta_z': 0.0}, {'delta_x': -65.80587005615234, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': 25.322040557861328, 'delta_z': 0.0}, {'delta_x': 65.80586242675781, 'delta_y': -25.322044372558594, 'delta_z': 0.0}]}, {'id': 'JDAP01760735938-1-1-', 'length': 14.0, 'width': 7.75, 'height': 4.05, 'weight': 0.7, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '252942', 'end_site_name': '天津津门湖营业部', 'predicted_alloc_time': '2025-04-22 10:20:15.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [2], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVA34452753071-1-1-', 'length': 17.1, 'width': 11.0, 'height': 8.1, 'weight': 1.62, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '124239', 'end_site_name': '天津水上公园营业部', 'predicted_alloc_time': '2025-04-22 10:19:04.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [6], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVA34463335425-1-1-', 'length': 17.35, 'width': 13.4, 'height': 11.7, 'weight': 2.92, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '1991529', 'end_site_name': '天津学府营业部', 'predicted_alloc_time': '2025-04-22 10:19:02.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [3], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVA34443215898-1-1-', 'length': 15.5, 'width': 13.4, 'height': 5.1, 'weight': 1.83, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '1991529', 'end_site_name': '天津学府营业部', 'predicted_alloc_time': '2025-04-22 10:18:25.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [3], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDAP20143649064-1-1-', 'length': 18.4, 'width': 14.25, 'height': 7.95, 'weight': 3.19, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '16', 'end_site_code': '252942', 'end_site_name': '天津津门湖营业部', 'predicted_alloc_time': '2025-04-22 10:09:45.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [2], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVC29876952241-1-1-', 'length': 13.2, 'width': 9.1, 'height': 8.45, 'weight': 2.47, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '1991529', 'end_site_name': '天津学府营业部', 'predicted_alloc_time': '2025-04-22 10:16:19.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [3], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDAZ19364779564-1-1-', 'length': 14.25, 'width': 9.35, 'height': 8.45, 'weight': 0.83, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '16', 'end_site_code': '124239', 'end_site_name': '天津水上公园营业部', 'predicted_alloc_time': '2025-04-22 10:15:55.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [6], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'xxxxxxxxxx-55', 'length': 15.0, 'width': 15.0, 'height': 15.0, 'weight': 5.0, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '16', 'end_site_code': 'site_xxx', 'end_site_name': '', 'predicted_alloc_time': '2025-04-22 10:15:23.098395', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [7, 8], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JD0186182422161-2-2-', 'length': 15.9, 'width': 14.95, 'height': 8.1, 'weight': 1.07, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '3032', 'end_site_name': '天津祥瑞营业部', 'predicted_alloc_time': '2025-04-22 10:18:19.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [4], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDAP01761525068-1-1-', 'length': 15.2, 'width': 10.25, 'height': 8.05, 'weight': 2.19, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '1412', 'end_site_name': '天津姚村营业部', 'predicted_alloc_time': '2025-04-22 10:16:58.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [5], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JD0186195915193-2-2-', 'length': 14.7, 'width': 13.05, 'height': 5.7, 'weight': 2.39, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '4188', 'end_site_name': '天津体北营业部', 'predicted_alloc_time': '2025-04-22 10:15:55.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [1], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JD0186191618900-1-1-', 'length': 19.0, 'width': 5.75, 'height': 12.6, 'weight': 0.68, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '124239', 'end_site_name': '天津水上公园营业部', 'predicted_alloc_time': '2025-04-22 10:15:52.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [6], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVA34443165157-1-1-', 'length': 15.7, 'width': 13.4, 'height': 4.95, 'weight': 1.66, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '252942', 'end_site_name': '天津津门湖营业部', 'predicted_alloc_time': '2025-04-22 10:16:28.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [2], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDAP01760979880-1-1-', 'length': 14.25, 'width': 6.45, 'height': 4.5, 'weight': 0.72, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '124239', 'end_site_name': '天津水上公园营业部', 'predicted_alloc_time': '2025-04-22 10:17:20.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [6], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JD0186175444757-1-1-', 'length': 11.7, 'width': 8.3, 'height': 12.75, 'weight': 1.37, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '3032', 'end_site_name': '天津祥瑞营业部', 'predicted_alloc_time': '2025-04-22 10:17:44.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [4], 'grasp_valid': [], 'grasp_points_delta': []}, {'id': 'JDVA34455020460-1-1-', 'length': 13.75, 'width': 11.85, 'height': 10.8, 'weight': 3.02, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '1412', 'end_site_name': '天津姚村营业部', 'predicted_alloc_time': '2025-04-22 10:17:22.000', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [5], 'grasp_valid': [], 'grasp_points_delta': []}], 'container': [{'pallet_id': 1, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 9.486067199707032, 'y': 29.952276611328124, 'z': 2.9312061309814452, 'length': 17.143182373046876, 'width': 19.161764526367186, 'height': 6.847190093994141, 'dw': 0.9999930262565613, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAWX0567327010-1-1-'}, {'x': 8.643296051025391, 'y': 29.62239990234375, 'z': 11.10931396484375, 'length': 17.218504333496092, 'width': 22.213465881347656, 'height': 10.353681945800782, 'dw': 0.9999778866767883, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34443490749-1-1-'}, {'x': 32.40864562988281, 'y': 30.97734375, 'z': 3.781351852416992, 'length': 15.016694641113281, 'width': 20.05937042236328, 'height': 8.078993225097657, 'dw': 0.999997079372406, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19347495398-1-1-'}, {'x': 25.05309295654297, 'y': 11.989899444580079, 'z': 4.26692008972168, 'length': 11.400409698486328, 'width': 15.15576629638672, 'height': 9.117170715332032, 'dw': 0.9998018145561218, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34465109739-1-1-'}, {'x': 26.675253295898436, 'y': 6.858721160888672, 'z': 12.548490142822265, 'length': 8.845211029052734, 'width': 12.018035125732421, 'height': 8.963700103759766, 'dw': -0.9945284128189087, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19352750609-1-1-'}, {'x': 8.790090179443359, 'y': 32.62922668457031, 'z': 21.36706085205078, 'length': 16.00867919921875, 'width': 17.525186157226564, 'height': 11.531350708007812, 'dw': -0.9995492696762085, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34447667781-1-1-'}, {'x': 9.713746643066406, 'y': 30.598605346679687, 'z': 30.883682250976562, 'length': 15.32024688720703, 'width': 17.326412963867188, 'height': 9.027962493896485, 'dw': 0.9998709559440613, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19332250059-1-1-'}, {'x': 7.8000022888183596, 'y': 30.1466796875, 'z': 40.499899291992186, 'length': 16.74891052246094, 'width': 19.496742248535156, 'height': 11.553500366210937, 'dw': 0.999963104724884, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038753739560-1-1-'}, {'x': 6.142642974853516, 'y': 3.551744079589844, 'z': 18.61492462158203, 'length': 18.10942840576172, 'width': 14.51357421875, 'height': 37.86496276855469, 'dw': -0.9991571307182312, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 35.23605651855469, 'y': 32.92127990722656, 'z': 13.063124084472657, 'length': 16.503804016113282, 'width': 20.020257568359376, 'height': 11.200773620605469, 'dw': -0.999925971031189, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 6.757423400878906, 'y': 14.942613220214843, 'z': 16.076797485351562, 'length': 20.211387634277344, 'width': 13.762728881835937, 'height': 32.635739135742185, 'dw': 0.9980452060699463, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 22.231365966796876, 'y': 4.095492553710938, 'z': 19.029324340820313, 'length': 14.424978637695313, 'width': 13.5728271484375, 'height': 5.303242111206055, 'dw': -0.9987313747406006, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 35.69015808105469, 'y': 5.990934753417969, 'z': 6.082985305786133, 'length': 15.206605529785156, 'width': 17.234075927734374, 'height': 13.034086608886719, 'dw': -0.9998535513877869, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 26.671920776367188, 'y': 16.251971435546874, 'z': 11.881723022460937, 'length': 20.4390380859375, 'width': 8.537071990966798, 'height': 7.306009674072266, 'dw': 0.9993200898170471, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 21.27992401123047, 'y': 30.141439819335936, 'z': 3.505300521850586, 'length': 9.031309509277344, 'width': 13.607878112792969, 'height': 7.845174407958984, 'dw': -0.9993934035301208, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 5.0009918212890625, 'y': 40.588690185546874, 'z': 36.41396484375, 'length': 14.816305541992188, 'width': 6.033631896972656, 'height': 19.69281463623047, 'dw': -0.9999082088470459, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29886028419-1-1-'}, {'x': 22.478741455078126, 'y': 34.390435791015626, 'z': 17.94228515625, 'length': 16.129679870605468, 'width': 11.22717514038086, 'height': 18.504937744140626, 'dw': -0.8751826882362366, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19355105243-1-1-'}, {'x': 30.130313110351562, 'y': 20.39091339111328, 'z': 26.709133911132813, 'length': 10.55920867919922, 'width': 18.740428161621093, 'height': 15.45814208984375, 'dw': -0.8670270442962646, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19355105243-1-1-'}, {'x': 20.39311981201172, 'y': 19.059088134765624, 'z': 24.756985473632813, 'length': 13.520843505859375, 'width': 16.388804626464843, 'height': 19.56720428466797, 'dw': -0.9984975457191467, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19355105243-1-1-'}], 'trans_matrix': [[-0.23741184175014496, 0.9713964462280273, -0.0049492428079247475, 190.5733184814453], [-0.971406877040863, -0.23739732801914215, 0.003354869317263365, 2555.30126953125], [0.0020836670882999897, 0.005604213569313288, 0.9999821186065674, -519.4509887695312], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 2, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 9.561180877685548, 'y': 31.830722045898437, 'z': 3.775138092041016, 'length': 16.566860961914063, 'width': 20.010948181152344, 'height': 8.235482788085937, 'dw': -0.9973524808883667, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40303420288-1-1-'}, {'x': 8.773532104492187, 'y': 12.323304748535156, 'z': 3.7873355865478517, 'length': 15.336378479003907, 'width': 20.213096618652344, 'height': 8.303437805175781, 'dw': 0.999954104423523, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34456415090-1-1-'}, {'x': 41.054010009765626, 'y': 33.5955078125, 'z': 2.7516153335571287, 'length': 12.427381896972657, 'width': 15.840696716308594, 'height': 6.1756744384765625, 'dw': -0.9997908473014832, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34439300134-1-1-'}, {'x': 38.06856994628906, 'y': 33.98822021484375, 'z': 8.553886413574219, 'length': 6.7545654296875, 'width': 13.218470764160156, 'height': 6.161783218383789, 'dw': 0.8975023031234741, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038687637648-1-1-'}, {'x': 41.89512329101562, 'y': 33.44926147460937, 'z': 10.51934356689453, 'length': 10.556973266601563, 'width': 15.688444519042969, 'height': 2.966268539428711, 'dw': 0.9998440742492676, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038687637648-1-1-'}, {'x': 40.372012329101565, 'y': 12.60889129638672, 'z': 5.31756362915039, 'length': 12.921023559570312, 'width': 18.138790893554688, 'height': 11.234037780761719, 'dw': 0.9997204542160034, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19360425818-1-1-'}, {'x': 9.095673370361329, 'y': 10.28248291015625, 'z': 11.571768951416015, 'length': 11.400835418701172, 'width': 15.360604858398437, 'height': 8.458841705322266, 'dw': 0.9987436532974243, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19353059056-1-1-'}, {'x': 26.80235595703125, 'y': 33.02708435058594, 'z': 3.8779781341552733, 'length': 12.918789672851563, 'width': 18.115631103515625, 'height': 8.580594635009765, 'dw': 0.9977378845214844, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07808041938-1-1-'}, {'x': 24.711061096191408, 'y': 11.328621673583985, 'z': 3.5112430572509767, 'length': 11.205848693847656, 'width': 15.321324157714844, 'height': 7.641197204589844, 'dw': 0.9999122619628906, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19340246989-1-1-'}, {'x': 8.915545654296874, 'y': 32.213751220703124, 'z': 15.219340515136718, 'length': 17.623651123046876, 'width': 22.629217529296874, 'height': 16.269070434570313, 'dw': 0.9978253841400146, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038632842356-1-1-'}, {'x': 28.821121215820312, 'y': 34.73997192382812, 'z': 12.067758178710937, 'length': 11.95265884399414, 'width': 16.428446960449218, 'height': 9.32869644165039, 'dw': -0.9975395202636719, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07792480711-1-1-'}, {'x': 10.645278167724609, 'y': 12.336080169677734, 'z': 20.855601501464843, 'length': 18.376374816894533, 'width': 19.19549560546875, 'height': 11.412064361572266, 'dw': -0.9996699690818787, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29917586422-1-1-'}, {'x': 40.934957885742186, 'y': 10.001589965820312, 'z': 13.452073669433593, 'length': 13.26807861328125, 'width': 16.083163452148437, 'height': 6.704972839355468, 'dw': 0.9993430972099304, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470019230-1-1-'}, {'x': 27.719525146484376, 'y': 32.797357177734376, 'z': 18.5943603515625, 'length': 13.675563049316406, 'width': 16.292706298828126, 'height': 6.375763320922852, 'dw': -0.9977423548698425, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470039686-1-1-'}, {'x': 41.0438232421875, 'y': 8.582633209228515, 'z': 18.97166290283203, 'length': 13.294888305664063, 'width': 16.245590209960938, 'height': 6.041569519042969, 'dw': -0.9993493556976318, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470035394-1-1-'}, {'x': 40.71210632324219, 'y': 9.96355743408203, 'z': 24.451742553710936, 'length': 12.885153198242188, 'width': 15.6855712890625, 'height': 6.212005615234375, 'dw': 0.9995976686477661, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470018393-1-1-'}, {'x': 9.898237609863282, 'y': 33.74308776855469, 'z': 26.423947143554688, 'length': 15.906721496582032, 'width': 16.37899169921875, 'height': 8.732231140136719, 'dw': -0.9939066171646118, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29875089384-1-1-'}], 'trans_matrix': [[0.41694432497024536, 0.9089195728302002, -0.004750045947730541, -1522.7430419921875], [-0.9088577628135681, 0.41683775186538696, -0.014962478540837765, 2312.99560546875], [-0.011619486846029758, 0.010555637069046497, 0.9998767971992493, -468.9011535644531], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 3, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 10.011948394775391, 'y': 42.996151733398435, 'z': 4.228833389282227, 'length': 17.774288940429688, 'width': 12.968267822265625, 'height': 9.39867706298828, 'dw': 0.9997526407241821, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34479322207-1-1-'}, {'x': 25.961624145507812, 'y': 42.00376892089844, 'z': 4.498188781738281, 'length': 11.74619140625, 'width': 15.864207458496093, 'height': 9.52246322631836, 'dw': 0.9949880242347717, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07804180541-1-1-'}, {'x': 10.38612060546875, 'y': 29.501162719726562, 'z': 4.1465911865234375, 'length': 16.361845397949217, 'width': 12.860556030273438, 'height': 8.80258560180664, 'dw': -0.9998521208763123, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34464730129-1-3-'}, {'x': 11.092820739746093, 'y': 43.290093994140626, 'z': 12.235687255859375, 'length': 19.24378204345703, 'width': 14.833074951171875, 'height': 8.1727294921875, 'dw': -0.9997262954711914, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34444572225-1-1-'}, {'x': 12.201435089111328, 'y': 8.987358856201173, 'z': 2.0768823623657227, 'length': 21.802760314941406, 'width': 16.990446472167967, 'height': 5.1382911682128904, 'dw': 0.9996634721755981, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0185951836759-1-1-'}, {'x': 12.381425476074218, 'y': 41.27330322265625, 'z': 21.151292419433595, 'length': 19.044520568847656, 'width': 16.757177734375, 'height': 10.575896453857421, 'dw': 0.9999956488609314, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVF03537594414-1-1-'}, {'x': 8.640853881835938, 'y': 6.039934158325195, 'z': 8.014886474609375, 'length': 13.832479858398438, 'width': 11.524868774414063, 'height': 8.304493713378907, 'dw': -0.9997899532318115, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29915733909-1-1-'}, {'x': 12.44234619140625, 'y': 6.595832824707031, 'z': 19.703175354003907, 'length': 19.963775634765625, 'width': 15.474790954589844, 'height': 16.113371276855467, 'dw': -0.9998921155929565, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34462123459-1-1-'}, {'x': 10.05151138305664, 'y': 18.721934509277343, 'z': 14.487713623046876, 'length': 17.546238708496094, 'width': 14.641242980957031, 'height': 21.583859252929688, 'dw': 0.9996203184127808, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34462123459-1-1-'}, {'x': 26.706353759765626, 'y': 39.353875732421876, 'z': 13.425694274902344, 'length': 16.851411437988283, 'width': 19.184193420410157, 'height': 10.179680633544923, 'dw': 0.9996929168701172, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34462123459-1-1-'}, {'x': 11.942903137207031, 'y': 30.6389892578125, 'z': 17.688900756835938, 'length': 15.258796691894531, 'width': 8.405072784423828, 'height': 19.13779296875, 'dw': 0.9996765851974487, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34462123459-1-1-'}, {'x': 8.210280609130859, 'y': 28.416183471679688, 'z': 17.969938659667967, 'length': 16.84279022216797, 'width': 11.881401062011719, 'height': 3.051035499572754, 'dw': -0.9981411695480347, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34462123459-1-1-'}, {'x': 50.530477905273436, 'y': 42.31587524414063, 'z': 2.696063423156738, 'length': 21.164581298828125, 'width': 17.596464538574217, 'height': 6.177788162231446, 'dw': 0.9998746514320374, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVF03538580949-1-1-'}, {'x': 49.32503051757813, 'y': 40.99688720703125, 'z': 12.797791290283204, 'length': 22.749958801269532, 'width': 18.09039764404297, 'height': 15.16840057373047, 'dw': -0.9998213052749634, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038715978513-1-2-'}, {'x': 35.36087646484375, 'y': 9.322587585449218, 'z': 7.533425903320312, 'length': 22.735546875, 'width': 17.74481658935547, 'height': 15.845565795898438, 'dw': -0.9999083280563354, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038715978513-2-2-'}, {'x': 48.852133178710936, 'y': 42.166802978515626, 'z': 22.941630554199218, 'length': 23.974143981933594, 'width': 17.94984130859375, 'height': 6.762656402587891, 'dw': -0.9995483756065369, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29875818294-1-1-'}, {'x': 47.98671875, 'y': 43.84097595214844, 'z': 29.820352172851564, 'length': 20.285969543457032, 'width': 14.959307861328124, 'height': 7.985697174072266, 'dw': -0.9999935030937195, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186171716768-1-1-'}, {'x': 53.1145751953125, 'y': 9.586721801757813, 'z': 5.742875671386718, 'length': 12.164038848876952, 'width': 19.383999633789063, 'height': 12.303105926513672, 'dw': -0.9999062418937683, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038712230655-1-1-'}, {'x': 34.90179443359375, 'y': 9.498001098632812, 'z': 17.794972229003907, 'length': 21.997645568847656, 'width': 17.582376098632814, 'height': 5.7363426208496096, 'dw': 0.9998387098312378, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34444270807-1-1-'}], 'trans_matrix': [[0.9104954600334167, 0.4135129153728485, -0.0022521757055073977, -2715.556396484375], [-0.41351717710494995, 0.9104612469673157, -0.00798382330685854, 772.6718139648438], [-0.0012508741347119212, 0.008200547657907009, 0.9999655485153198, -556.7974243164062], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 4, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 11.263356018066407, 'y': 46.1039306640625, 'z': 4.059553146362305, 'length': 20.662403869628907, 'width': 9.977082061767579, 'height': 8.640222930908203, 'dw': 0.9998243451118469, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29914885683-1-1-'}, {'x': 28.733029174804688, 'y': 44.94473571777344, 'z': 3.1630144119262695, 'length': 15.139802551269531, 'width': 11.093795013427734, 'height': 7.276585388183594, 'dw': 0.9990292191505432, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20144301421-1-1-'}, {'x': 46.94186706542969, 'y': 44.32286376953125, 'z': 3.948015594482422, 'length': 19.50009765625, 'width': 13.763323974609374, 'height': 8.52896957397461, 'dw': -0.9998228549957275, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34458585629-1-1-'}, {'x': 44.10817260742188, 'y': 45.6982177734375, 'z': 10.889781951904297, 'length': 12.152848815917968, 'width': 9.381239318847657, 'height': 6.582289123535157, 'dw': -0.9994913935661316, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19362368250-1-1-'}, {'x': 9.751998901367188, 'y': 9.867993927001953, 'z': 5.23475227355957, 'length': 18.714535522460938, 'width': 18.314785766601563, 'height': 11.171593475341798, 'dw': -0.9993078112602234, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34453105953-1-1-'}, {'x': 25.193147277832033, 'y': 8.759038543701172, 'z': 3.294245147705078, 'length': 10.160453796386719, 'width': 13.351901245117187, 'height': 7.276610565185547, 'dw': -0.9998679757118225, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19340360067-1-1-'}, {'x': 39.62715148925781, 'y': 11.162892913818359, 'z': 8.578235626220703, 'length': 16.330914306640626, 'width': 18.825128173828126, 'height': 17.902890014648438, 'dw': -0.9992299675941467, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV020624868646-1-1-'}, {'x': 10.126374816894531, 'y': 32.79679565429687, 'z': 2.986343002319336, 'length': 20.09552917480469, 'width': 14.274760437011718, 'height': 6.728736114501953, 'dw': -0.9991765022277832, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20144325956-1-1-'}, {'x': 6.529170227050781, 'y': 8.231121826171876, 'z': 14.240921020507812, 'length': 16.81482849121094, 'width': 14.444107055664062, 'height': 8.31076431274414, 'dw': 0.9998299479484558, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV020641721274-1-1-'}, {'x': 8.541895294189453, 'y': 32.5216064453125, 'z': 9.134475708007812, 'length': 16.309788513183594, 'width': 13.170700073242188, 'height': 7.540280151367187, 'dw': -0.9987466931343079, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19357831000-1-1-'}, {'x': 38.93750915527344, 'y': 15.429856872558593, 'z': 25.171896362304686, 'length': 19.261631774902344, 'width': 25.818609619140624, 'height': 17.59605255126953, 'dw': 0.9975878596305847, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20143677312-1-1-'}, {'x': 10.959190368652344, 'y': 32.38540344238281, 'z': 16.1741455078125, 'length': 19.586105346679688, 'width': 13.450672912597657, 'height': 9.136354064941406, 'dw': -0.9992154836654663, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470925835-1-1-'}, {'x': 11.12131576538086, 'y': 35.88153076171875, 'z': 24.355197143554687, 'length': 20.851901245117187, 'width': 15.90760498046875, 'height': 9.486991882324219, 'dw': 0.9944344758987427, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20143945197-1-1-'}, {'x': 10.001382446289062, 'y': 9.688562774658203, 'z': 23.365707397460938, 'length': 20.00933074951172, 'width': 15.147950744628906, 'height': 11.422372436523437, 'dw': 0.9995255470275879, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP01742145458-1-1-'}], 'trans_matrix': [[0.8982729911804199, -0.4388371407985687, -0.022971050813794136, -2310.247802734375], [0.43802332878112793, 0.898348331451416, -0.03325841948390007, -1583.2691650390625], [0.035231005400419235, 0.019813288003206253, 0.9991828203201294, -579.908447265625], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 5, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 10.047903442382813, 'y': 7.895559692382813, 'z': 3.1596675872802735, 'length': 20.219082641601563, 'width': 8.641624450683594, 'height': 7.173862457275391, 'dw': 0.9992042779922485, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19348382907-1-1-'}, {'x': 9.911398315429688, 'y': 18.993498229980467, 'z': 3.0019760131835938, 'length': 20.455943298339843, 'width': 8.485753631591797, 'height': 6.818024444580078, 'dw': -0.9999639987945557, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19348382916-1-1-'}, {'x': 8.595416259765624, 'y': 35.821755981445314, 'z': 4.74154052734375, 'length': 18.257174682617187, 'width': 13.974436950683593, 'height': 10.465364837646485, 'dw': -0.9999585747718811, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19350342713-1-1-'}, {'x': 10.822884368896485, 'y': 35.874386596679685, 'z': 15.141929626464844, 'length': 19.107688903808594, 'width': 13.175424194335937, 'height': 11.050958251953125, 'dw': 0.9998798370361328, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34470912560-1-1-'}, {'x': 9.751377868652344, 'y': 24.215678405761718, 'z': 11.084976196289062, 'length': 17.285638427734376, 'width': 12.652100372314454, 'height': 10.915289306640625, 'dw': 0.9817250967025757, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07808494150-1-1-'}, {'x': 8.596060943603515, 'y': 35.65475158691406, 'z': 24.209327697753906, 'length': 15.429518127441407, 'width': 11.884843444824218, 'height': 8.277471923828125, 'dw': 0.9998700022697449, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20144752301-1-1-'}, {'x': 7.706687927246094, 'y': 35.663211059570315, 'z': 31.56185302734375, 'length': 10.012061309814452, 'width': 13.390103149414063, 'height': 7.600890350341797, 'dw': -0.9993677735328674, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19339889614-1-1-'}, {'x': 7.516880035400391, 'y': 12.41344985961914, 'z': 12.392793273925781, 'length': 17.38026580810547, 'width': 12.570799255371094, 'height': 12.647418212890624, 'dw': -0.9997932314872742, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34473488521-1-1-'}, {'x': 9.964073944091798, 'y': 34.332342529296874, 'z': 39.38056335449219, 'length': 15.759330749511719, 'width': 17.482012939453124, 'height': 9.551244354248047, 'dw': 0.9996287226676941, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19349684944-1-1-'}, {'x': 34.001852416992186, 'y': 32.2126953125, 'z': 7.4434349060058596, 'length': 22.012313842773438, 'width': 17.495619201660155, 'height': 15.735696411132812, 'dw': -0.999991774559021, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34439322229-1-1-'}, {'x': 34.40804443359375, 'y': 9.55910873413086, 'z': 6.388508605957031, 'length': 21.533352661132813, 'width': 13.00595703125, 'height': 13.328225708007812, 'dw': -0.9996173977851868, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038740769617-1-1-'}, {'x': 35.70826416015625, 'y': 36.09505920410156, 'z': 19.472999572753906, 'length': 17.431117248535156, 'width': 14.27391357421875, 'height': 9.70296859741211, 'dw': 0.9956124424934387, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29893742319-1-1-'}, {'x': 11.15517349243164, 'y': 34.0089599609375, 'z': 47.534524536132814, 'length': 18.532962036132812, 'width': 15.789338684082031, 'height': 7.790845489501953, 'dw': -0.9999469518661499, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19327701241-1-1-'}, {'x': 31.749661254882813, 'y': 19.18201904296875, 'z': 7.595150756835937, 'length': 22.713803100585938, 'width': 10.936708068847656, 'height': 15.724948120117187, 'dw': -0.9998686909675598, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19346728661-1-1-'}, {'x': 34.779153442382814, 'y': 3.3550872802734375, 'z': 17.27862091064453, 'length': 17.404634094238283, 'width': 10.533577728271485, 'height': 9.745897674560547, 'dw': -0.9995854496955872, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19346728661-1-1-'}, {'x': 35.01361999511719, 'y': 33.72001342773437, 'z': 30.318023681640625, 'length': 19.06181640625, 'width': 16.193304443359374, 'height': 13.639506530761718, 'dw': -0.9999718070030212, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0185929541482-1-1-'}], 'trans_matrix': [[0.470632404088974, -0.8820666670799255, 0.021527912467718124, -877.8980712890625], [0.8823203444480896, 0.47037774324417114, -0.015986550599336624, -2631.77587890625], [0.003974883351475, 0.026518307626247406, 0.9996404647827148, -488.9874267578125], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 6, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 9.337050628662109, 'y': 35.988739013671875, 'z': 3.8330554962158203, 'length': 20.688081359863283, 'width': 10.915106964111327, 'height': 8.628341674804688, 'dw': 0.9995291829109192, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07815579149-1-1-'}, {'x': 11.639180755615234, 'y': 24.264772033691408, 'z': 3.6027965545654297, 'length': 19.625883483886717, 'width': 15.300692749023437, 'height': 8.139822387695313, 'dw': -0.99992835521698, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038717428144-1-1-'}, {'x': 8.518417358398438, 'y': 30.351651000976563, 'z': 15.818186950683593, 'length': 26.659915161132812, 'width': 22.226712036132813, 'height': 17.831675720214843, 'dw': -0.8970999717712402, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34479086772-1-1-'}, {'x': 9.018327331542968, 'y': 5.363186264038086, 'z': 3.2970882415771485, 'length': 16.367848205566407, 'width': 8.66463623046875, 'height': 7.565433502197266, 'dw': -0.9998757243156433, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34473224077-1-1-'}, {'x': 35.85628051757813, 'y': 33.246026611328126, 'z': 10.773847961425782, 'length': 16.14824676513672, 'width': 16.9909912109375, 'height': 21.87825927734375, 'dw': 0.9904438853263855, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19349529722-1-1-'}, {'x': 36.031393432617186, 'y': 8.939722442626953, 'z': 2.8040493011474608, 'length': 22.131977844238282, 'width': 17.202244567871094, 'height': 6.540152740478516, 'dw': 0.9998751282691956, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVF03538553876-1-1-'}, {'x': 37.346945190429686, 'y': 8.771400451660156, 'z': 10.808394622802734, 'length': 17.432759094238282, 'width': 15.126416015625, 'height': 10.60910873413086, 'dw': 0.9999984502792358, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34449589938-1-1-'}, {'x': 10.638681030273437, 'y': 5.941405487060547, 'z': 10.23266830444336, 'length': 17.360935974121094, 'width': 8.938196563720703, 'height': 7.346752166748047, 'dw': -0.9990750551223755, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV020618939032-1-1-'}, {'x': 37.029989624023436, 'y': 9.850148010253907, 'z': 19.246078491210938, 'length': 17.720616149902344, 'width': 14.598764038085937, 'height': 7.551585388183594, 'dw': 0.9994733333587646, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC29888410817-1-1-'}, {'x': 33.815798950195315, 'y': 9.317294311523437, 'z': 27.24010009765625, 'length': 20.298724365234374, 'width': 14.209446716308594, 'height': 9.639405822753906, 'dw': 0.9998666048049927, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA34459315809-1-1-'}], 'trans_matrix': [[-0.22130908071994781, -0.9751949906349182, 0.004119100049138069, 956.7118530273438], [0.9751753807067871, -0.2213328629732132, -0.006680500227957964, -2383.849609375], [0.0074268062599003315, 0.002538389293476939, 0.9999691843986511, -493.84002685546875], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 7, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 19.984693908691405, 'y': 27.337322998046876, 'z': 1.1195881843566895, 'length': 13.985479736328125, 'width': 8.447101593017578, 'height': 2.756393623352051, 'dw': -0.9976074695587158, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '399.693878'}, {'x': 34.2376220703125, 'y': 37.432211303710936, 'z': 1.4689481735229493, 'length': 25.185708618164064, 'width': 12.48617172241211, 'height': 3.5563106536865234, 'dw': -0.9995755553245544, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '684.752441'}, {'x': 40.785589599609374, 'y': 13.783856201171876, 'z': 1.8638151168823243, 'length': 11.821215057373047, 'width': 33.31849365234375, 'height': 4.430535888671875, 'dw': 0.9955831170082092, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '815.711792'}], 'trans_matrix': [[0.026035500690340996, -0.9996498823165894, 0.00472600944340229, 1541.644287109375], [0.9987014532089233, 0.02621731534600258, 0.04368247464299202, -1468.277099609375], [-0.04379107058048248, 0.0035825904924422503, 0.9990341067314148, -649.453125], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 8, 'end_machine_code': 'TJBCSHFJ-CZ-ZDFJJ-SZJL-2LD009', 'end_chute_code': '18', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'package_list': [{'x': 31.462921142578125, 'y': 18.161767578125, 'z': 4.837697601318359, 'length': 15.289207458496094, 'width': 15.690898132324218, 'height': 7.322208404541016, 'dw': 0.7416427135467529, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '629.258423'}, {'x': 29.802569580078124, 'y': 18.840245056152344, 'z': 4.850518417358399, 'length': 19.109825134277344, 'width': 19.811073303222656, 'height': 11.342890167236328, 'dw': 0.8152914047241211, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '596.051392'}], 'trans_matrix': [[0.022343777120113373, -0.9994465708732605, -0.024644751101732254, 1563.4727783203125], [0.9997211694717407, 0.022524623200297356, -0.007084935437887907, 521.5325927734375], [0.00763619365170598, -0.024479573592543602, 0.9996711611747742, -640.0226440429688], [0.0, 0.0, 0.0, 1.0]]}], 'rotate_angle': 90.0, 'is_trajectory_error': 0, 'is_break_in': 0}

In [109]:
import numpy as np

In [186]:
 a = [{'x': 8.304785919189452, 'y': 23.176272583007812, 'z': 27.623153686523438, 'length': 18.64905700683594, 'width': 13.55401153564453, 'height': 55.4099365234375, 'dw': -0.9963807463645935, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '166.095718'}, {'x': 31.11259765625, 'y': 34.53934326171875, 'z': 15.29108123779297, 'length': 37.141561889648436, 'width': 42.61494750976563, 'height': 5.609759140014648, 'dw': 0.9997550845146179, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '622.251953'}, {'x': 11.15710220336914, 'y': 6.2942039489746096, 'z': 23.659686279296874, 'length': 17.054463195800782, 'width': 17.030303955078125, 'height': 44.49403686523438, 'dw': 0.9614425301551819, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '223.142044'}, {'x': 38.54178466796875, 'y': 39.302365112304685, 'z': 24.196649169921876, 'length': 16.199366760253906, 'width': 19.32500457763672, 'height': 14.81470947265625, 'dw': 0.9714670181274414, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '770.835693'}, {'x': 23.754780578613282, 'y': 43.07252197265625, 'z': 24.886752319335937, 'length': 14.649856567382812, 'width': 25.404151916503906, 'height': 15.140531921386719, 'dw': 0.9293184876441956, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '475.095612'}, {'x': 10.404660034179688, 'y': 33.651971435546876, 'z': 40.719906616210935, 'length': 18.73517761230469, 'width': 13.089083862304687, 'height': 46.511697387695314, 'dw': 0.9986026883125305, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '208.093201'}, {'x': 9.0923095703125, 'y': 8.135617065429688, 'z': 64.54707641601563, 'length': 15.841441345214843, 'width': 22.708676147460938, 'height': 50.17193603515625, 'dw': -0.9988911151885986, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '181.846191'}, {'x': 35.355307006835936, 'y': 15.9317626953125, 'z': 33.302825927734375, 'length': 23.05882568359375, 'width': 7.232810974121094, 'height': 1.9613679885864257, 'dw': -0.9965205788612366, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '707.106140'}, {'x': 8.256295013427735, 'y': 6.778954315185547, 'z': 92.77893676757813, 'length': 11.556327819824219, 'width': 18.125900268554688, 'height': 7.831002807617187, 'dw': 0.9969639778137207, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '165.125900'}, {'x': 15.245991516113282, 'y': 8.478315734863282, 'z': 78.99368286132812, 'length': 7.025830078125, 'width': 21.386456298828126, 'height': 3.9678951263427735, 'dw': -0.9418203234672546, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '304.919830'}, {'x': 19.552369689941408, 'y': 46.10304565429688, 'z': 68.93329467773438, 'length': 5.000949096679688, 'width': 11.776890563964844, 'height': 2.3163280487060547, 'dw': -0.9492926597595215, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '391.047394'}, {'x': 30.754974365234375, 'y': 26.4556396484375, 'z': 33.95721435546875, 'length': 16.849632263183594, 'width': 21.89421844482422, 'height': 12.689404296875, 'dw': 0.8953284025192261, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 36.76130676269531, 'y': 45.31656494140625, 'z': 36.99545593261719, 'length': 16.222662353515624, 'width': 19.272398376464842, 'height': 15.16533203125, 'dw': -0.9978348016738892, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 38.882073974609376, 'y': 25.25902557373047, 'z': 38.326953125, 'length': 13.775384521484375, 'width': 19.399537658691408, 'height': 2.3546480178833007, 'dw': -0.9993979334831238, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 29.575424194335938, 'y': 16.977093505859376, 'z': 33.99547119140625, 'length': 25.824798583984375, 'width': 40.96177368164062, 'height': 5.08564338684082, 'dw': 0.9988824725151062, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 24.12584228515625, 'y': 44.33827209472656, 'z': 34.97569580078125, 'length': 13.131787109375, 'width': 17.47122802734375, 'height': 7.269618988037109, 'dw': -0.9999123215675354, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 11.255644226074219, 'y': 47.107452392578125, 'z': 68.22108764648438, 'length': 19.706675720214843, 'width': 16.0270263671875, 'height': 2.8199489593505858, 'dw': -0.9978424310684204, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB40245599181-1-1-'}, {'x': 8.53202133178711, 'y': 43.0372314453125, 'z': 74.38026123046875, 'length': 13.7376220703125, 'width': 21.64507598876953, 'height': 12.136454772949218, 'dw': 0.9977429509162903, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDK002227039447-1-1-'}, {'x': 7.065218353271485, 'y': 41.274148559570314, 'z': 84.28101196289063, 'length': 11.266951751708984, 'width': 15.214424133300781, 'height': 9.947358703613281, 'dw': 0.9986099004745483, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD07793395120-1-1-'}, {'x': 10.04347915649414, 'y': 43.07893371582031, 'z': 92.74328002929687, 'length': 16.91991271972656, 'width': 21.294148254394532, 'height': 9.56064682006836, 'dw': -0.9983729720115662, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186018543563-1-1-'}, {'x': 9.860176849365235, 'y': 26.133447265625, 'z': 72.47427978515626, 'length': 17.513897705078126, 'width': 17.11118927001953, 'height': 18.484239196777345, 'dw': 0.9994015693664551, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186018543563-1-1-'}, {'x': 35.23818664550781, 'y': 6.493663787841797, 'z': 39.6148681640625, 'length': 23.409278869628906, 'width': 19.970111083984374, 'height': 11.169234466552734, 'dw': 0.9974445104598999, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186018543563-1-1-'}, {'x': 10.632819366455077, 'y': 8.328633880615234, 'z': 56.870477294921876, 'length': 15.00171356201172, 'width': 13.495028686523437, 'height': 15.522428894042969, 'dw': 0.7749367356300354, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186018543563-1-1-'}, {'x': 31.002029418945312, 'y': 6.979754638671875, 'z': 47.81510620117187, 'length': 17.200469970703125, 'width': 13.322671508789062, 'height': 9.301297760009765, 'dw': 0.9951189756393433, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19350553375-1-1-'}, {'x': 7.924220275878906, 'y': 25.595303344726563, 'z': 98.02234497070313, 'length': 13.159010314941407, 'width': 22.0244873046875, 'height': 2.6187017440795897, 'dw': 0.9989389181137085, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19350553375-1-1-'}, {'x': 28.753372192382812, 'y': 4.995965957641602, 'z': 53.1175537109375, 'length': 21.69817352294922, 'width': 14.944837951660157, 'height': 6.048415756225586, 'dw': -0.9901986122131348, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19350553375-1-1-'}, {'x': 13.643408203125, 'y': 23.399778747558592, 'z': 84.54972534179687, 'length': 6.820152282714844, 'width': 10.134781646728516, 'height': 7.374141693115234, 'dw': 0.9787243008613586, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ19350553375-1-1-'}, {'x': 38.299139404296874, 'y': 7.158383178710937, 'z': 50.756524658203126, 'length': 8.627064514160157, 'width': 15.285379028320312, 'height': 11.282469940185546, 'dw': -0.922682523727417, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186038029913-1-1-'}, {'x': 34.1808837890625, 'y': 12.752767944335938, 'z': 54.6039306640625, 'length': 14.331668090820312, 'width': 4.863718795776367, 'height': 3.6256595611572267, 'dw': -0.9836737513542175, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186038029913-1-1-'}, {'x': 29.297418212890626, 'y': 6.4519798278808596, 'z': 64.318603515625, 'length': 29.642636108398438, 'width': 18.752232360839844, 'height': 16.7297119140625, 'dw': 0.9990293383598328, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038677020000-1-1-'}, {'x': 22.255339050292967, 'y': 4.628042221069336, 'z': 80.90806274414062, 'length': 14.453875732421874, 'width': 14.489474487304687, 'height': 4.301795578002929, 'dw': 0.999101996421814, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038677020000-1-1-'}, {'x': 25.51624450683594, 'y': 27.24128112792969, 'z': 39.628302001953124, 'length': 16.018896484375, 'width': 18.803303527832032, 'height': 4.025979232788086, 'dw': -0.9993579387664795, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038677020000-1-1-'}, {'x': 22.837220764160158, 'y': 13.216899108886718, 'z': 62.413079833984376, 'length': 12.929953002929688, 'width': 4.794772338867188, 'height': 2.5115358352661135, 'dw': 0.9602599143981934, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX038677020000-1-1-'}, {'x': 31.846978759765626, 'y': 42.719906616210935, 'z': 48.63232421875, 'length': 13.911640930175782, 'width': 20.63373260498047, 'height': 9.856561279296875, 'dw': 0.9955875277519226, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186011560619-1-1-'}, {'x': 29.943209838867187, 'y': 27.50544128417969, 'z': 47.803466796875, 'length': 23.618753051757814, 'width': 16.722291564941408, 'height': 5.698831176757812, 'dw': 0.999083399772644, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186024923978-1-2-'}, {'x': 10.182676696777344, 'y': 43.55122375488281, 'z': 99.34778442382813, 'length': 18.029164123535157, 'width': 16.308567810058594, 'height': 6.302421569824219, 'dw': -0.9992451071739197, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186024923978-1-2-'}, {'x': 42.137078857421876, 'y': 45.52424011230469, 'z': 49.078488159179685, 'length': 10.783089447021485, 'width': 12.888076782226562, 'height': 11.103398895263672, 'dw': -0.9966548085212708, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186024923978-1-2-'}, {'x': 29.964602661132812, 'y': 27.7745361328125, 'z': 56.57661743164063, 'length': 20.93858184814453, 'width': 16.496881103515626, 'height': 7.9654388427734375, 'dw': -0.9979594945907593, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0186024923978-2-2-'}]

In [187]:
len(a)

38

In [188]:
a[35]

{'x': 10.182676696777344,
 'y': 43.55122375488281,
 'z': 99.34778442382813,
 'length': 18.029164123535157,
 'width': 16.308567810058594,
 'height': 6.302421569824219,
 'dw': -0.9992451071739197,
 'dx': 1.0,
 'dy': 1.0,
 'dz': 1.0,
 'package_code': 'JD0186024923978-1-2-'}

In [526]:
max([abs(i['delta_x']) for i in input_dict['package'][0]['grasp_points_delta']])

121.00660705566406

In [499]:
len([True, False, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False])

100

In [1273]:
output_dict['sku_stack_positions'][0][9][0]

[[-0.7210543751716614,
  -0.692878246307373,
  -0.000515617139171809,
  1713.8788819841357],
 [-0.6928777694702148,
  0.7210525870323181,
  0.0018697914201766253,
  -1556.0614022186808],
 [-0.0009237499325536191,
  0.001705452799797058,
  -0.9999980926513672,
  1288.918961275342],
 [0.0, 0.0, 0.0, 1.0]]

In [1776]:
input_dict = {'package': [{'id': 'JDAZ21669541409-1-1-', 'length': 18.915316772460937, 'width': 12.424661254882812, 'height': 7.75, 'weight': 1.22, 'end_machine_code': 'BJXM-CZ-ZDFJJ-38-SZJL-LC14', 'end_chute_code': '38', 'end_site_code': '1372', 'end_site_name': '北京龙旺庄站', 'predicted_alloc_time': '2025-10-20 08:54:35.161', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': True, 'pallet_id': [2], 'grasp_valid': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'grasp_points_delta': [{'delta_x': 63.47035217285156, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': 63.47035217285156, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': -63.470340728759766, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': 38.08219909667969, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -38.08220672607422, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -12.694068908691406, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': 12.694061279296875, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': 88.85847473144531, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': -88.85847473144531, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': 1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': -1.0170183181762695, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': 3.051055908203125, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': -3.051055908203125, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': 5.0850934982299805, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': -5.085093021392822, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': -7.1191301345825195, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': 7.119131088256836, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': -9.153167724609375, 'delta_z': 0.0}, {'delta_x': 114.24661254882812, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': 9.153167724609375, 'delta_z': 0.0}, {'delta_x': -114.24661254882812, 'delta_y': -9.153167724609375, 'delta_z': 0.0}]}], 'container': [{'pallet_id': 1, 'end_machine_code': 'BJXM-CZ-ZDFJJ-38-SZJL-LC14', 'end_chute_code': '38', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'pallet_type': 's', 'is_closed': 0, 'package_list': [{'x': 41.8324951171875, 'y': 31.147073364257814, 'z': 4.409347152709961, 'length': 10.155459594726562, 'width': 15.320242309570313, 'height': 9.576871490478515, 'dw': -0.9997674226760864, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAWX0837770142-1-1-'}, {'x': 11.240655517578125, 'y': 6.090383911132813, 'z': 4.8742424011230465, 'length': 17.844410705566407, 'width': 12.245038604736328, 'height': 10.514099884033204, 'dw': -0.9995980858802795, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAWX0837769241-1-1-'}, {'x': 40.343502807617185, 'y': 30.583786010742188, 'z': 13.396044921875, 'length': 10.46400146484375, 'width': 14.728096008300781, 'height': 9.226719665527344, 'dw': 0.9998219013214111, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ21681171434-1-1-'}, {'x': 31.822549438476564, 'y': 7.8870086669921875, 'z': 5.773181915283203, 'length': 23.339015197753906, 'width': 14.850967407226562, 'height': 12.338995361328125, 'dw': -0.9999018907546997, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX044029908122-1-1-'}, {'x': 32.012228393554686, 'y': 8.2723388671875, 'z': 14.172566223144532, 'length': 22.27858123779297, 'width': 15.062281799316406, 'height': 5.1225746154785154, 'dw': -0.9999654293060303, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ21675979009-1-1-'}, {'x': 9.009952545166016, 'y': 29.184344482421874, 'z': 4.529523849487305, 'length': 17.859761047363282, 'width': 22.1419921875, 'height': 9.6556640625, 'dw': 0.9999852776527405, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX044038506371-1-1-'}, {'x': 8.212242889404298, 'y': 29.004739379882814, 'z': 13.420396423339843, 'length': 16.939732360839844, 'width': 17.941058349609374, 'height': 9.678758239746093, 'dw': -0.9966529607772827, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38775542341-1-1-'}, {'x': 7.161595153808594, 'y': 29.2839111328125, 'z': 22.513883972167967, 'length': 14.878970336914062, 'width': 17.51014404296875, 'height': 12.395191192626953, 'dw': 0.9925220012664795, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVC32972847077-1-1-'}, {'x': 31.35211181640625, 'y': 8.736615753173828, 'z': 19.9233642578125, 'length': 21.411056518554688, 'width': 14.771255493164062, 'height': 7.551414489746094, 'dw': 0.9992976784706116, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDX044041074469-1-1-'}, {'x': 66.3483642578125, 'y': 23.116183471679687, 'z': 2.832517623901367, 'length': 30.634744262695314, 'width': 24.70997314453125, 'height': 6.522163391113281, 'dw': -0.9999228715896606, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38824625973-1-1-'}, {'x': 18.353152465820312, 'y': 30.23193359375, 'z': 12.971110534667968, 'length': 3.850697326660156, 'width': 16.651284790039064, 'height': 8.356037139892578, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38824625973-1-1-'}, {'x': 74.82786865234375, 'y': 1.5834821701049804, 'z': 3.078146743774414, 'length': 6.785543823242188, 'width': 4.164056777954102, 'height': 7.017756652832031, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38821985015-1-1-'}, {'x': 32.07465515136719, 'y': 8.912568664550781, 'z': 27.10284423828125, 'length': 21.75914306640625, 'width': 17.079879760742188, 'height': 8.245889282226562, 'dw': 0.9999822378158569, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38821985015-1-1-'}, {'x': 10.408095550537109, 'y': 6.8506309509277346, 'z': 12.456806945800782, 'length': 13.288998413085938, 'width': 9.948226928710938, 'height': 6.051739501953125, 'dw': -0.9995208978652954, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280181769-1-1-'}, {'x': 10.107266235351563, 'y': 5.932032775878906, 'z': 19.568023681640625, 'length': 14.709135437011719, 'width': 10.64952392578125, 'height': 9.248056793212891, 'dw': 0.9998448491096497, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20279955623-1-1-'}, {'x': 27.652847290039062, 'y': 30.574398803710938, 'z': 17.653298950195314, 'length': 21.880888366699217, 'width': 21.293531799316405, 'height': 21.173063659667967, 'dw': 0.9403588175773621, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280259926-1-1-'}, {'x': 10.85552978515625, 'y': 6.932987213134766, 'z': 26.994805908203126, 'length': 13.061431884765625, 'width': 9.200686645507812, 'height': 7.321241760253907, 'dw': 0.9998322129249573, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280259926-1-1-'}], 'trans_matrix': [[0.008532947860658169, -0.9999353885650635, -0.0074982051737606525, 389.9273681640625], [0.9999600648880005, 0.008552567102015018, -0.0025892097037285566, -2524.62841796875], [0.0026531561743468046, -0.007475811056792736, 0.999968409538269, -337.4315185546875], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 2, 'end_machine_code': 'BJXM-CZ-ZDFJJ-38-SZJL-LC14', 'end_chute_code': '38', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'pallet_type': 's', 'is_closed': 0, 'package_list': [{'x': 7.7640541076660154, 'y': 30.401473999023438, 'z': 3.6480453491210936, 'length': 12.858865356445312, 'width': 15.997357177734376, 'height': 7.877796936035156, 'dw': 0.9996951222419739, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38804177066-1-1-'}, {'x': 8.055567932128906, 'y': 30.319866943359376, 'z': 10.389662933349609, 'length': 11.805386352539063, 'width': 13.41317138671875, 'height': 7.169090270996094, 'dw': -0.9983519911766052, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ21654990351-1-1-'}, {'x': 9.519322967529297, 'y': 28.859588623046875, 'z': 16.916934204101562, 'length': 15.200540161132812, 'width': 16.124179077148437, 'height': 6.841205596923828, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV023497597608-1-1-'}, {'x': 26.184915161132814, 'y': 30.723239135742187, 'z': 3.8968994140625, 'length': 13.019692993164062, 'width': 15.746604919433594, 'height': 8.467359161376953, 'dw': -0.996667206287384, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVD08824633589-1-1-'}, {'x': 11.915454864501953, 'y': 9.742487335205078, 'z': 8.176114654541015, 'length': 20.063449096679687, 'width': 19.196424865722655, 'height': 17.144764709472657, 'dw': 0.9999076724052429, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB50365367959-1-1-'}, {'x': 41.2724609375, 'y': 30.519558715820313, 'z': 4.1100421905517575, 'length': 10.793409729003907, 'width': 14.886695861816406, 'height': 9.16446533203125, 'dw': -0.9990637898445129, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAZ21679017786-1-1-'}, {'x': 26.905044555664062, 'y': 30.453585815429687, 'z': 12.60464859008789, 'length': 10.20672607421875, 'width': 16.213145446777343, 'height': 11.258174896240234, 'dw': -0.9962383508682251, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38819925687-1-1-'}, {'x': 11.784618377685547, 'y': 10.434352111816406, 'z': 18.720114135742186, 'length': 18.31757354736328, 'width': 18.481051635742187, 'height': 4.888783645629883, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD0211770315852-1-1-'}, {'x': 12.128551483154297, 'y': 29.4048828125, 'z': 22.46605224609375, 'length': 17.43659210205078, 'width': 12.484403991699219, 'height': 5.529911041259766, 'dw': -0.9987571239471436, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280015663-1-1-'}, {'x': 12.416815185546875, 'y': 29.428173828125, 'z': 26.977294921875, 'length': 17.872805786132812, 'width': 12.350189971923829, 'height': 6.043430328369141, 'dw': -0.9978300333023071, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280023516-1-1-'}, {'x': 18.720559692382814, 'y': 13.424276733398438, 'z': 24.082823181152342, 'length': 29.074957275390624, 'width': 21.566473388671874, 'height': 9.233094787597656, 'dw': 0.9931084513664246, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38808123918-1-1-'}, {'x': 19.30638427734375, 'y': 13.158090209960937, 'z': 32.5385986328125, 'length': 29.484176635742188, 'width': 21.69914093017578, 'height': 14.591790771484375, 'dw': 0.9943782687187195, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38810242940-1-1-'}, {'x': 72.16458129882812, 'y': 25.8982177734375, 'z': 1.6785003662109375, 'length': 5.145420074462891, 'width': 7.271968841552734, 'height': 3.9680927276611326, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38827309855-1-1-'}, {'x': 65.07252807617188, 'y': 28.820254516601562, 'z': 2.3256101608276367, 'length': 11.038686370849609, 'width': 14.420802307128906, 'height': 5.330758285522461, 'dw': 0.9864262342453003, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38827309855-1-1-'}, {'x': 54.25218505859375, 'y': 30.372738647460938, 'z': 2.523290824890137, 'length': 7.47112808227539, 'width': 14.364816284179687, 'height': 6.036355972290039, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38827309855-1-1-'}, {'x': 73.5439208984375, 'y': 33.64405212402344, 'z': 4.813557815551758, 'length': 14.322299194335937, 'width': 15.319657897949218, 'height': 6.196509933471679, 'dw': -0.9832538962364197, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38827309855-1-1-'}, {'x': 34.568783569335935, 'y': 30.216873168945312, 'z': 4.7049304962158205, 'length': 3.7845977783203124, 'width': 13.240643310546876, 'height': 10.328787994384765, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38827309855-1-1-'}, {'x': 40.83780212402344, 'y': 10.409517669677735, 'z': 3.2001316070556642, 'length': 11.206517791748047, 'width': 16.796878051757812, 'height': 7.206669616699219, 'dw': -0.9971123337745667, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDAP20280292065-1-1-'}, {'x': 18.964620971679686, 'y': 13.311909484863282, 'z': 41.566891479492185, 'length': 29.119430541992188, 'width': 21.366432189941406, 'height': 10.368536376953125, 'dw': -0.994398295879364, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVB50348284079-1-1-'}, {'x': 16.680397033691406, 'y': 10.183197021484375, 'z': 49.516622924804686, 'length': 22.516261291503906, 'width': 17.317857360839845, 'height': 11.40536880493164, 'dw': -0.9897189140319824, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDVA38818104772-1-1-'}, {'x': 34.28071899414063, 'y': 16.033505249023438, 'z': 21.422413635253907, 'length': 2.564888763427734, 'width': 7.679911804199219, 'height': 43.379946899414065, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV023521600217-1-1-'}, {'x': 25.943658447265626, 'y': 11.264146423339843, 'z': 52.30521240234375, 'length': 10.356385803222656, 'width': 5.647195434570312, 'height': 7.51732177734375, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV023521600217-1-1-'}, {'x': 14.0566162109375, 'y': 5.823442459106445, 'z': 55.79132080078125, 'length': 5.129595184326172, 'width': 11.768632507324218, 'height': 4.881004714965821, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV023521600217-1-1-'}, {'x': 26.832379150390626, 'y': 4.657566070556641, 'z': 60.849725341796876, 'length': 7.171551513671875, 'width': 6.210879898071289, 'height': 18.590660095214844, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JDV023521600217-1-1-'}], 'trans_matrix': [[-0.692878246307373, -0.7210543751716614, 0.000515617139171809, 1994.512939453125], [0.7210525870323181, -0.6928777694702148, -0.0018697914201766253, -1573.8472900390625], [0.001705452799797058, -0.0009237499325536191, 0.9999980926513672, -331.2619934082031], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 3, 'end_machine_code': 'BJXM-CZ-ZDFJJ-38-SZJL-LC14', 'end_chute_code': '38', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'pallet_type': 'n', 'is_closed': 0, 'package_list': [], 'trans_matrix': [[-0.9321834444999695, 0.3616505563259125, 0.015584778040647507, 2330.1884765625], [-0.36140841245651245, -0.9322641491889954, 0.01635664701461792, 1354.01416015625], [0.02044435404241085, 0.00961492769420147, 0.9997446537017822, -405.0562438964844], [0.0, 0.0, 0.0, 1.0]]}, {'pallet_id': 4, 'end_machine_code': 'BJXM-CZ-ZDFJJ-38-SZJL-LC14', 'end_chute_code': '38', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'pallet_type': 'e', 'is_closed': 0, 'package_list': [{'x': 34.414251708984374, 'y': -0.5940651893615723, 'z': 1.6826305389404297, 'length': 8.722706604003907, 'width': 3.87291259765625, 'height': 4.352517318725586, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '688.285034'}, {'x': 25.440081787109374, 'y': -0.6533864498138428, 'z': 1.870961570739746, 'length': 9.100057220458984, 'width': 3.7960636138916017, 'height': 4.708697128295898, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '508.801636'}, {'x': -1.2255769729614259, 'y': -0.7974196910858155, 'z': 1.8769981384277343, 'length': 5.71863021850586, 'width': 5.253866195678711, 'height': 4.752359390258789, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '-24.511539'}, {'x': -2.8902910232543944, 'y': 26.76006164550781, 'z': 2.382850456237793, 'length': 7.2356201171875, 'width': 5.266928482055664, 'height': 5.7545928955078125, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '-57.805820'}, {'x': 46.834173583984374, 'y': 37.72611083984375, 'z': 2.3480064392089846, 'length': 37.757684326171876, 'width': 24.740855407714843, 'height': 6.2199146270751955, 'dw': 0.9934422373771667, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '936.683472'}, {'x': 51.00001220703125, 'y': 48.01192016601563, 'z': 15.31016845703125, 'length': 28.119790649414064, 'width': 5.2153667449951175, 'height': 20.69084777832031, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '1020.000244'}, {'x': -2.818842124938965, 'y': -2.4464725494384765, 'z': 16.12376708984375, 'length': 6.759957885742187, 'width': 5.788765335083008, 'height': 24.543601989746094, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '-56.376842'}, {'x': 12.967469787597656, 'y': 22.316368103027344, 'z': 1.0642569541931153, 'length': 17.931239318847656, 'width': 6.416097259521484, 'height': 3.125900459289551, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '259.349396'}, {'x': 24.140901184082033, 'y': 28.694549560546875, 'z': 1.5584101676940918, 'length': 7.683859252929688, 'width': 5.025993347167969, 'height': 4.098731994628906, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '482.818024'}, {'x': 43.02981872558594, 'y': 12.299874877929687, 'z': 2.303909683227539, 'length': 9.221659851074218, 'width': 27.176638793945312, 'height': 5.605684661865235, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '860.596375'}, {'x': 24.052365112304688, 'y': -2.146711730957031, 'z': 16.10937042236328, 'length': 8.31219253540039, 'width': 2.830532264709473, 'height': 24.307400512695313, 'dw': 1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '481.047302'}, {'x': 53.592340087890626, 'y': 45.04355163574219, 'z': 57.564605712890625, 'length': 11.779139709472656, 'width': 5.946209335327149, 'height': 65.7244384765625, 'dw': -1.0, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': '1071.846802'}], 'trans_matrix': [[-0.9999343156814575, -0.008379481732845306, -0.007822900079190731, 1208.6036376953125], [0.008376140147447586, -0.9999648332595825, 0.00045957634574733675, 1360.0811767578125], [-0.007826427929103374, 0.00039402119000442326, 0.9999693632125854, -460.88031005859375], [0.0, 0.0, 0.0, 1.0]]}], 'rotate_angle': 90.0, 'is_trajectory_error': 0, 'is_break_in': 0, 'is_rotate_package': 0, 'package_num': 0}

output_dict = {'pallet_id': 4, 'sku_stack_positions': [[0, 21, 69, 21, 37, 78, 'JDVB50753322348-1-1-', [1], [[[-0.6769611835479736, -0.7359107732772827, 0.012608179822564125, -2352.7704675808536], [-0.7360039353370667, 0.6769569516181946, -0.00524343503639102, 2419.064239301278], [-0.004676495678722858, -0.012829319573938847, -0.9999066591262817, 749.5291209617665], [0.0, 0.0, 0.0, 1.0]]], [[[-0.6769611835479736, -0.7359107732772827, 0.012608179822564125, -2352.7704675808536], [-0.7360039353370667, 0.6769569516181946, -0.00524343503639102, 2419.064239301278], [-0.004676495678722858, -0.012829319573938847, -0.9999066591262817, 749.5291209617665], [0.0, 0.0, 0.0, 1.0]]]]], 'sku_stack_position_priority': [0], 'full': False, 'full_pallet_id': [], 'full_pallet_id_remove': [], 'full_pallet_id_adjust': [], 'full_pallet_id_closed': [], 'rotate_flag': 0}

#{'pallet_id': 4, 'sku_stack_positions': [[3, 0, 0, 20, 13, 9, 'JDAP20244016006-1-1-', [61, 62, 65, 66, 69, 70, 73, 74, 77, 78, 81, 83, 86, 88, 90, 92, 94, 96, 97, 99], [[[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -543.3421180828124], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2506.466936720174], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.2832909929777], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -543.3421106121807], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2506.4669385374655], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.28329105834825], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -542.973436202557], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2503.252557124344], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.30744876862843], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -542.9734437818552], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2503.2525562544897], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.30744869613744], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -542.6047619015998], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2500.0381766586593], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.3316064717882], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -542.604769480898], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2500.0381757888044], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.3316063992972], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -542.2360874919761], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2496.8237952455383], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.3557641820684], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -542.2360949626078], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2496.823793428247], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.35576411669786], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -541.8674206616506], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2493.6094129625626], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.3799218198576], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -541.8674130823524], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2493.609413832417], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.37992189234865], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -517.7577953098427], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2509.4031774171985], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.52798822304874], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -517.7577802599128], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2509.4031801043443], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.5279883609103], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -517.3891210088855], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2506.1887969515137], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.5521459262085], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -517.3891058502891], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2506.1887986912234], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.5521460711905], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -517.0204467079283], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2502.974416485829], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.5763036293682], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -517.0204315493319], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2502.9744182255386], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.57630377435027], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -516.6517721896381], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2499.7600341252714], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.60046134676895], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -516.6517571397083], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2499.760036812417], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.6004614846305], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -516.2830827300845], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2496.5456553992963], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.6246191949107], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -516.2830978886809], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2496.5456536595866], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.6246190499287], [0.0, 0.0, 0.0, 1.0]]], [[[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -553.2764558179884], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2505.326800746482], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.1882755826438], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -553.2764483473568], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2505.3268025637735], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.1882756480144], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -552.907773937733], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2502.112421150652], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.21243335829456], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -552.9077815170313], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2502.112420280798], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.21243328580357], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -552.5390996367759], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2498.8980406849673], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.2365910614543], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -552.5391072160741], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2498.8980398151125], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.2365909889633], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -552.1704252271522], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2495.6836592718464], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.26074877173454], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -552.1704326977839], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2495.683657454555], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.260748706364], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -551.8017583968267], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2492.4692769888707], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.2849064095237], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -551.8017508175285], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2492.469277858725], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.2849064820148], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -527.6921330450187], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2508.2630414435066], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.43297281271487], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -527.6921179950889], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2508.2630441306524], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.4329729505764], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -527.3234587440616], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2505.048660977822], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.4571305158746], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -527.3234435854652], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2505.0486627175314], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.45713066085665], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -526.9547844431044], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2501.834280512137], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.48128821903435], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -526.954769284508], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2501.8342822518466], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.4812883640164], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -526.5861099248142], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2498.6198981515795], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.5054459364351], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -526.5860948748843], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2498.6199008387252], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.50544607429663], [0.0, 0.0, 0.0, 1.0]], [[0.9934337735176086, 0.11394506692886353, -0.010290662758052349, -526.2174204652606], [0.11401359736919403, -0.9934589862823486, 0.006334678735584021, 2495.4055194256043], [-0.009501541033387184, -0.00746634928509593, -0.9999269247055054, -434.5296037845768], [0.0, 0.0, 0.0, 1.0]], [[-0.9934337735176086, -0.11394506692886353, -0.010290662758052349, -526.217435623857], [-0.11401359736919403, 0.9934589862823486, 0.006334678735584021, 2495.4055176858947], [0.009501541033387184, 0.00746634928509593, -0.9999269247055054, -434.5296036395948], [0.0, 0.0, 0.0, 1.0]]]]], 'sku_stack_position_priority': [0], 'full': False, 'full_pallet_id': [], 'full_pallet_id_remove': [], 'full_pallet_id_adjust': [], 'full_pallet_id_closed': []}


container_package_list = [item for item in input_dict['container'] if item['pallet_id']==output_dict['pallet_id']][0]['package_list']

log_packages = []
log_packages = [
    [p['length'], p['width'], p['height'], p['x']-p['length']/2, p['y']-p['width']/2, p['z']-p['height']/2] for p in container_package_list
]

this_container = {
    'length': 47,
    'width': 40,
    'height': 80
}
# i=0
# for placed_package in container_package_list:
#     if placed_package['length'] == 0 or placed_package['width'] == 0:
#         continue
#     lx = max(0, placed_package['x'] - placed_package['length'] / 2)
#     rx = min(this_container['length'], placed_package['x'] + placed_package['length'] / 2)
#     ly = max(0, placed_package['y'] - placed_package['width'] / 2)
#     ry = min(this_container['width'], placed_package['y'] + placed_package['width'] / 2)
#     lz = max(0, placed_package['z'] - placed_package['height'] / 2)
#     if math.ceil(lx) >= this_container['length'] or math.ceil(rx - lx) <= 0 or math.ceil(ly) >= this_container['width'] or math.ceil(ry - ly) <= 0:
#         continue
#     if 0.4 < lz < 1:
#         lz = 0
#     rz = min(this_container['height'], placed_package['z'] + placed_package['height'] / 2)
#     if (rx - lx <= 0.5) or (ry - ly <= 0.5) or (rz - lz <= 0.5):
#         continue
#     else:
#         i+=1
#         # if i in [6,7,8]:
#         print(i)
#         print(placed_package)
#         print(lx, ly, lz, rx, ry, rz)
#         print([math.ceil(rx - lx), math.ceil(ry - ly), round(rz - lz), math.floor(lx), math.floor(ly), round(lz)])
#         log_packages.append([math.ceil(rx - lx), math.ceil(ry - ly), round(rz - lz), math.floor(lx), math.floor(ly), round(lz)])

# 这个是包裹取整到容器后的：
# output_position = {
#     'x1': output_dict['sku_stack_positions'][0][0],
#     'y1': output_dict['sku_stack_positions'][0][1],
#     'z1': output_dict['sku_stack_positions'][0][2],
#     'x2': output_dict['sku_stack_positions'][0][3],
#     'y2': output_dict['sku_stack_positions'][0][4],
#     'z2': output_dict['sku_stack_positions'][0][5],
# }
# 这个是用原始包裹的尺寸
output_package = [item for item in input_dict['package'] if item['id']==output_dict['sku_stack_positions'][0][6]][0]
if output_dict['sku_stack_positions'][0][4]-output_dict['sku_stack_positions'][0][1]>output_dict['sku_stack_positions'][0][3]-output_dict['sku_stack_positions'][0][0]:
    output_package['length'], output_package['width'] = output_package['width'], output_package['length']
    
output_position = {
    'x1': (output_dict['sku_stack_positions'][0][0]+output_dict['sku_stack_positions'][0][3]-output_package['length'])/2,
    'y1': (output_dict['sku_stack_positions'][0][1]+output_dict['sku_stack_positions'][0][4]-output_package['width'])/2,
    'z1': (output_dict['sku_stack_positions'][0][2]+output_dict['sku_stack_positions'][0][5]-output_package['height'])/2,
    'x2': (output_dict['sku_stack_positions'][0][0]+output_dict['sku_stack_positions'][0][3]+output_package['length'])/2,
    'y2': (output_dict['sku_stack_positions'][0][1]+output_dict['sku_stack_positions'][0][4]+output_package['width'])/2,
    'z2': (output_dict['sku_stack_positions'][0][2]+output_dict['sku_stack_positions'][0][5]+output_package['height'])/2,
}


# output_position = {
#     'x1': output_dict['sku_stack_positions'][0][0],
#     'y1': output_dict['sku_stack_positions'][0][1],
#     'z1': output_dict['sku_stack_positions'][0][2],
#     'x2': output_dict['sku_stack_positions'][0][3],
#     'y2': output_dict['sku_stack_positions'][0][4],
#     'z2': output_dict['sku_stack_positions'][0][5],
# }
log_packages.append([
    output_position['x2']-output_position['x1']-2, output_position['y2']-output_position['y1']-2, output_position['z2']-output_position['z1'], output_position['x1']+1, output_position['y1']+1, output_position['z1']
])

# log_packages.append([
#     output_position['x2']-output_position['x1'], output_position['y2']-output_position['y1'], output_position['z2']-output_position['z1'], output_position['x1'], output_position['y1'], output_position['z1']
# ])

A = [item for item in input_dict['container'] if item['pallet_id']==output_dict['pallet_id']][0]['trans_matrix']
C = output_dict['sku_stack_positions'][0][8][0]
print(C)
A_inv = np.linalg.inv(A)
B = np.dot(A_inv, C)

log_packages.append([
    16, 16, 1, B[0][3]/20-17/2, B[1][3]/20-17/2, B[2][3]/20
])

# # log_packages.append([
# #     16, 16, 1, B[0][3]/20-17/2, B[1][3]/20-17/2, B[2][3]/20
# # ])

plot_cube_plotly(log_packages,[47,40,80])#,[47,40,80]) #[47,30,40])#,[47,40,80]) [47,30,40]
print('finished! ')#[47,30,40]

[[0.692878246307373, -0.7210543751716614, -0.000515617139171809, 1686.6448672370166], [-0.7210525870323181, -0.6928777694702148, 0.0018697914201766253, -1530.906019950913], [-0.001705452799797058, -0.0009237499325536191, -0.9999980926513672, 988.9794816119334], [0.0, 0.0, 0.0, 1.0]]
finished! 


In [ ]:
29, 26, 44, 47, 39, 52

In [13]:
str_list = [[66.31859, 549.4415,513.0641,304.61404,440.31158,1042.1774],
[422.76852, 548.17944,227.72841,418.98904,395.21872,444.39752],
[64.06401, 188.71605,796.757,258.41187,463.87695,852.2787],
[436.45084, 64.894646,98.275314,248.26074,203.63211,210.00465],
[484.26373, 279.32764,263.4926,401.41174,410.91144,527.5148],
[216.15393, 316.2976,380.58374,132.6016,274.488,83.520935],
[542.0988, 721.1596,88.7146,203.43912,174.01428,158.81377],
[599.7568, 783.6576,121.924675,252.61465,148.08112,44.428986],
[575.4734, 351.91302,309.09528,273.33936,225.31902,72.66071],
[661.8955, 571.77386,220.56776,400.57483,347.5893,280.5675],
[188.06604, 765.2151,106.96792,139.20828,209.97,225.56564],
[197.01677, 676.1954,821.59766,109.612045,235.65942,58.92827],
[403.36136, 183.99641,597.8445,351.96075,251.8467,197.75922],
[434.11578, 81.73901,765.39716,376.5279,275.4667,225.17723],
[596.27893, 415.93683,90.58385,170.14565,148.07745,195.89044]]

In [18]:
dict = {}
count = 1
for item in str_list:
    x = item[0]
    y = item[1]
    z = item[2]
    len = item[3]
    wid = item[4]
    height = item[5]
    dict["x"] = x
    dict["y"] = y
    dict["z"] = z
    dict["length"] = len
    dict["width"] = wid
    dict["height"] = height
    dict["dw"] = 0.11
    dict["dx"] = 1
    dict["dy"] = 1
    dict["dz"] = 1
    dict["package_code"] = str(count)
    count+=1
    # print(item)
    # print(item[0])
    print(dict)

{'x': 66.31859, 'y': 549.4415, 'z': 513.0641, 'length': 304.61404, 'width': 440.31158, 'height': 1042.1774, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package_code': '1'}
{'x': 422.76852, 'y': 548.17944, 'z': 227.72841, 'length': 418.98904, 'width': 395.21872, 'height': 444.39752, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package_code': '2'}
{'x': 64.06401, 'y': 188.71605, 'z': 796.757, 'length': 258.41187, 'width': 463.87695, 'height': 852.2787, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package_code': '3'}
{'x': 436.45084, 'y': 64.894646, 'z': 98.275314, 'length': 248.26074, 'width': 203.63211, 'height': 210.00465, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package_code': '4'}
{'x': 484.26373, 'y': 279.32764, 'z': 263.4926, 'length': 401.41174, 'width': 410.91144, 'height': 527.5148, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package_code': '5'}
{'x': 216.15393, 'y': 316.2976, 'z': 380.58374, 'length': 132.6016, 'width': 274.488, 'height': 83.520935, 'dw': 0.11, 'dx': 1, 'dy': 1, 'dz': 1, 'package

In [12]:
dict

{'x': '[',
 'y': '2',
 'z': '0',
 'length': '2',
 'width': '4',
 'height': '-',
 'dw': 0.11,
 'dx': 1,
 'dy': 1,
 'dz': 1,
 'package_code': '15'}

In [1736]:
a = [{'delta_x': -54.61550521850586, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -54.61550521850586, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 54.61549377441406, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 10.923103332519531, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -10.923103332519531, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 76.46170043945312, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -76.46170806884766, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -98.30790710449219, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 98.30790710449219, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': -31.45379638671875, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': 31.45379638671875, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': -24.464065551757812, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': 24.46406364440918, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': -17.47433090209961, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': 17.47433090209961, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': 10.484598159790039, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': -10.484600067138672, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}, {'delta_x': -32.76930236816406, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': -3.4948654174804688, 'delta_z': 0.0}, {'delta_x': 32.76930236816406, 'delta_y': 3.4948654174804688, 'delta_z': 0.0}]

In [1737]:
a[29]

{'delta_x': -98.30790710449219, 'delta_y': 31.45379638671875, 'delta_z': 0.0}

In [1767]:
input_dict

{'pallet_id': 2,
 'sku_stack_positions': [[26,
   28,
   0,
   40,
   40,
   11,
   'JD0211353782910-1-1-',
   [33],
   [[[0.21907265484333038,
      -0.9756800532341003,
      0.00745282182469964,
      1814.3401006855781],
     [-0.9757034182548523,
      -0.21908950805664062,
      -0.001515332842245698,
      199.56865305638803],
     [0.003111316356807947,
      -0.006939688231796026,
      -0.9999710321426392,
      -362.73268253950937],
     [0.0, 0.0, 0.0, 1.0]]],
   [[[0.21907265484333038,
      -0.9756800532341003,
      0.00745282182469964,
      1826.2876277663524],
     [-0.9757034182548523,
      -0.21908950805664062,
      -0.001515332842245698,
      192.0025139544059],
     [0.003111316356807947,
      -0.006939688231796026,
      -0.9999710321426392,
      -362.63217249362333],
     [0.0, 0.0, 0.0, 1.0]]]]],
 'sku_stack_position_priority': [0],
 'full': False,
 'full_pallet_id': [],
 'full_pallet_id_remove': [],
 'full_pallet_id_adjust': [],
 'full_pallet_id_closed': 

In [1868]:
input_dict = {'package': [{'id': 'JD641', 'length': 12.223824310302735, 'width': 9.274566650390625, 'height': 10.41864013671875, 'weight': 1.0, 'end_machine_code': 'BJXM-CZ-ZDFJJ-SZJL-LA16', 'end_chute_code': '33', 'end_site_code': 'site_xxx', 'end_site_name': 'site_xxx', 'predicted_alloc_time': '2025-11-05 16:10:01.619753', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': True, 'pallet_id': [1], 'grasp_valid': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False], 'grasp_points_delta': [{'delta_x': -40.41426086425781, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 82.47090911865234, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 67.41273498535156, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 97.5290756225586, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 52.35456848144531, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 112.58724212646484, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 37.29640197753906, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 22.238235473632812, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 127.64541625976562, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 142.70358276367188, 'delta_z': 0.0}, {'delta_x': 40.41426086425781, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': -40.41426086425781, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': 24.248558044433594, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': -24.248554229736328, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': 56.5799560546875, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': -56.57996368408203, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': 8.082855224609375, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': -8.082855224609375, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': 72.74566650390625, 'delta_y': 157.76174926757812, 'delta_z': 0.0}, {'delta_x': -72.74566650390625, 'delta_y': 157.76174926757812, 'delta_z': 0.0}]}, {'id': 'JD491', 'length': 2.5, 'width': 2.5, 'height': 0.5, 'weight': 1.0, 'end_machine_code': 'BJXM-CZ-ZDFJJ-SZJL-LA16', 'end_chute_code': '33', 'end_site_code': 'site_xxx', 'end_site_name': 'site_xxx', 'predicted_alloc_time': '2025-11-05 16:10:01.619828', 'alloc_time': '', 'pac_type': '', 'cons_name': '', 'tracked': False, 'pallet_id': [1], 'grasp_valid': [], 'grasp_points_delta': []}], 'container': [{'pallet_id': 1, 'end_machine_code': 'BJXM-CZ-ZDFJJ-SZJL-LA16', 'end_chute_code': '16', 'end_site_code': '', 'end_site_name': '', 'first_flag': False, 'pallet_type': 's', 'is_closed': 0, 'package_list': [{'x': 11.18151626586914, 'y': 10.749139404296875, 'z': 7.615121459960937, 'length': 21.745802307128905, 'width': 17.17920379638672, 'height': 15.76331787109375, 'dw': 0.999876856803894, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD176'}, {'x': 6.932437896728516, 'y': 30.69902648925781, 'z': 4.889994812011719, 'length': 14.640480041503906, 'width': 16.395947265625, 'height': 10.306339263916016, 'dw': -0.999904990196228, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD90'}, {'x': 12.123301696777343, 'y': 11.62489013671875, 'z': 20.58997039794922, 'length': 22.311911010742186, 'width': 19.18714294433594, 'height': 12.249834442138672, 'dw': -0.9918544292449951, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD233'}, {'x': 11.917506408691406, 'y': 11.875760650634765, 'z': 30.986648559570312, 'length': 21.612026977539063, 'width': 18.202267456054688, 'height': 12.889311218261719, 'dw': 0.9934979677200317, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD472'}, {'x': 38.456207275390625, 'y': 31.2319580078125, 'z': 4.8908226013183596, 'length': 14.230029296875, 'width': 16.932550048828126, 'height': 10.73904037475586, 'dw': 0.9998653531074524, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD784'}, {'x': 12.945027160644532, 'y': 10.376662445068359, 'z': 41.87977905273438, 'length': 22.176423645019533, 'width': 19.14795837402344, 'height': 12.686290740966797, 'dw': 0.9902290105819702, 'dx': 1.0, 'dy': 1.0, 'dz': 1.0, 'package_code': 'JD30'}], 'trans_matrix': [[0.9997705221176147, 0.00984449964016676, 0.019027121365070343, 429.8058776855469], [-0.009642443619668484, 0.9998964071273804, -0.010681957006454468, -2328.569580078125], [-0.019130311906337738, 0.01049603708088398, 0.9997618794441223, -1673.6767578125], [0.0, 0.0, 0.0, 0.9999999403953552]]}], 'rotate_angle': 90.0, 'is_trajectory_error': 0, 'is_break_in': 0, 'is_rotate_package': 0, 'package_num': 0}

output_dict = {'pallet_id': 1, 'sku_stack_positions': [[25, 12, 0, 38, 22, 10, 'JD641', [1], [[[0.00984449964016676, 0.9997705221176147, -0.019027121365070343, 1133.8132678474603], [0.9998964071273804, -0.009642443619668484, 0.010681957006454468, -2037.8760302677217], [0.01049603708088398, -0.019130311906337738, -0.9997618794441223, -1483.9216420445246], [0.0, 0.0, 0.0, 0.9999999403953552]]], [[[0.00984449964016676, 0.9997705221176147, -0.019027121365070343, 1133.8132678474603], [0.9998964071273804, -0.009642443619668484, 0.010681957006454468, -2037.8760302677217], [0.01049603708088398, -0.019130311906337738, -0.9997618794441223, -1483.9216420445246], [0.0, 0.0, 0.0, 0.9999999403953552]]]]], 'sku_stack_position_priority': [0], 'full': False, 'full_pallet_id': [], 'full_pallet_id_remove': [], 'full_pallet_id_adjust': [], 'full_pallet_id_closed': [], 'rotate_flag': 0}

# {'pallet_id': 2, 'sku_stack_positions': [[26, 28, 0, 40, 40, 11, 'JD0211353782910-1-1-', [33], [[[0.21907265484333038, -0.9756800532341003, 0.00745282182469964, 1814.3401006855781], [-0.9757034182548523, -0.21908950805664062, -0.001515332842245698, 199.56865305638803], [0.003111316356807947, -0.006939688231796026, -0.9999710321426392, -362.73268253950937], [0.0, 0.0, 0.0, 1.0]]], [[[0.21907265484333038, -0.9756800532341003, 0.00745282182469964, 1826.2876277663524], [-0.9757034182548523, -0.21908950805664062, -0.001515332842245698, 192.0025139544059], [0.003111316356807947, -0.006939688231796026, -0.9999710321426392, -362.63217249362333], [0.0, 0.0, 0.0, 1.0]]]]], 'sku_stack_position_priority': [0], 'full': False, 'full_pallet_id': [], 'full_pallet_id_remove': [], 'full_pallet_id_adjust': [], 'full_pallet_id_closed': [], 'rotate_flag': 0}

is_buffer = 1

package_buffer = 1

container_package_list = [item for item in input_dict['container'] if item['pallet_id']==output_dict['pallet_id']][0]['package_list']

log_packages = []
log_packages = [
    [p['length'], p['width'], p['height'], p['x']-p['length']/2, p['y']-p['width']/2, p['z']-p['height']/2] for p in container_package_list
]

# output_position = {
#     'x1': output_dict['sku_stack_positions'][0][0],
#     'y1': output_dict['sku_stack_positions'][0][1],
#     'z1': output_dict['sku_stack_positions'][0][2],
#     'x2': output_dict['sku_stack_positions'][0][3],
#     'y2': output_dict['sku_stack_positions'][0][4],
#     'z2': output_dict['sku_stack_positions'][0][5],
# }
output_package = [item for item in input_dict['package'] if item['id']==output_dict['sku_stack_positions'][0][6]][0]

if output_dict['sku_stack_positions'][0][4]-output_dict['sku_stack_positions'][0][1]>output_dict['sku_stack_positions'][0][3]-output_dict['sku_stack_positions'][0][0]:
    if output_package['length']>output_package['width']:
        output_package['length'], output_package['width'] = output_package['width'], output_package['length']

output_position = {
    'x1': (output_dict['sku_stack_positions'][0][0]+output_dict['sku_stack_positions'][0][3]-output_package['length'])/2,
    'y1': (output_dict['sku_stack_positions'][0][1]+output_dict['sku_stack_positions'][0][4]-output_package['width'])/2,
    'z1': (output_dict['sku_stack_positions'][0][2]+output_dict['sku_stack_positions'][0][5]-output_package['height'])/2,
    'x2': (output_dict['sku_stack_positions'][0][0]+output_dict['sku_stack_positions'][0][3]+output_package['length'])/2,
    'y2': (output_dict['sku_stack_positions'][0][1]+output_dict['sku_stack_positions'][0][4]+output_package['width'])/2,
    'z2': (output_dict['sku_stack_positions'][0][2]+output_dict['sku_stack_positions'][0][5]+output_package['height'])/2,
}

if is_buffer:
    log_packages.append([
        output_position['x2']-output_position['x1'], output_position['y2']-output_position['y1'], output_position['z2']-output_position['z1'], output_position['x1'], output_position['y1'], output_position['z1']
    ])
else:
    log_packages.append([
        output_position['x2']-output_position['x1']-package_buffer, output_position['y2']-output_position['y1']-package_buffer, output_position['z2']-output_position['z1'], output_position['x1']+package_buffer/2, output_position['y1']+package_buffer/2, output_position['z1']
    ])

A = [item for item in input_dict['container'] if item['pallet_id']==output_dict['pallet_id']][0]['trans_matrix']
C = output_dict['sku_stack_positions'][0][8][0]
A_inv = np.linalg.inv(A)
B = np.dot(A_inv, C)
log_packages.append([
    17, 17, 1, B[0][3]/20-17/2, B[1][3]/20-17/2, B[2][3]/20
])

plot_cube_plotly(log_packages, [47, 40, 80])
# plot_cube_plotly(log_packages, [60, 50, 60])
print('finished! ')

finished! 


In [1570]:
output_dict['sku_stack_positions'][0][9][0]

[[-0.35916709899902344,
  0.9331763386726379,
  0.013455979526042938,
  -1003.2315911102415],
 [0.9332138895988464,
  0.35926955938339233,
  -0.006097327917814255,
  3192.272287127825],
 [-0.010523959062993526,
  0.010367348790168762,
  -0.9998908042907715,
  733.3140008564505],
 [0.0, 0.0, 0.0, 1.0]]

In [1571]:
output_dict['sku_stack_positions'][0][8][0]

[[-0.35916709899902344,
  0.9331763386726379,
  0.013455979526042938,
  -996.0482491302611],
 [0.9332138895988464,
  0.35926955938339233,
  -0.006097327917814255,
  3173.6080093358482],
 [-0.010523959062993526,
  0.010367348790168762,
  -0.9998908042907715,
  733.5244800377104],
 [0.0, 0.0, 0.0, 1.0]]